In [170]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import LatentDirichletAllocation
import unicodedata
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SaidKalee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SaidKalee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SaidKalee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Importing the data

In [171]:
#importing all the data
df_KVK2022 = pd.read_excel('data/data22.xlsx') #data 2022 from KVK requested SBI numbers
df_KVK2023 = pd.read_excel('data/data23.xlsx') #data 2023 KVK
df_monitor = pd.read_excel('data/Resultaat.xlsx') #data based on KVK numbers from monitor
df_label = pd.read_excel('data/data_sbi_select.xlsx') #random selection manually labeled to check if ZZG or not
df_adres = pd.read_excel('data/ZZG_NAW_JanSmelik.xlsx') #adresdata from ZZG (self-identified)

In [172]:
df_KVK2022

,DOSSIER,VGNUMMER,HN1X30,STRVA,PCPLVA,STRCA,PCPLCA,HN1X2X30,HN2X2X30,HN45,...,W_P_TOTAAL,W_P_PARTT,WP_TOT_OND,H_NAAM_VOL,IND_OPHEFF,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label
0,32130500,21836205,Scouting Ernst van Kempengroep,van der Helstlaan 2,3741 RT BAARN,Van Neslaan 87,3742 MP BAARN,Scouting Ernst van Kempengroep,NaN,Scouting Ernst van Kempengroep,...,0,0,0,Scouting Ernst van Kempengroep,N,Scoutingvereniging,2011,1,24,0
1,76721841,44476957,Coöp. Duurz. Gortel Niersen UA,Oranjeweg 118,8166 JD EMST,Oranjeweg 118,8166 JD EMST,Coöperatie Duurzaam Gortel,Niersen UA,Coöperatie Duurzaam Gortel Niersen UA,...,0,0,0,Coöperatie Duurzaam Gortel Niers,N,Het bijdragen aan de verduurzaming van de same...,2019,12,24,1
2,20052809,22777164,CVVE Winkelcentr. de Gagel UA,Auvergnestraat 7,4611 LL BERGEN OP ZOOM,Postbus 350,4600 AJ BERGEN OP ZOOM,CVVE,Winkelcentr. de Gagel UA,CVVE Winkelcentr. de Gagel UA,...,0,0,0,Coöperatieve Vereniging van Eige,N,Het behartigen van de gemeenschappelijke belan...,2013,11,29,0
3,77157419,44876009,Coöperatie Nieuwe Energie UA,Kievitmeen 30,3844 XG HARDERWIJK,Kievitmeen 30,3844 XG HARDERWIJK,Coöperatie Nieuwe Energie UA,NaN,Coöperatie Nieuwe Energie UA,...,0,0,0,Coöperatie Nieuwe Energie UA,N,Het voorzien in de stoffelijke behoeften van h...,2020,1,28,0
4,85151742,37656791,De Verbindingskamer,Deijffelbroekseweg 47,3195 GM PERNIS ROTTERDAM,Deijffelbroekseweg 47,3195 GM PERNIS ROTTERDAM,De Verbindingskamer,NaN,De Verbindingskamer,...,1,0,1,De Verbindingskamer,N,Adviesburo op het gebied van welzijn en sociaa...,2022,1,12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,65953525,34623329,Maris coaching,Oranje Nassaulaan 51,2051 HJ OVERVEEN,Oranje Nassaulaan 51,2051 HJ OVERVEEN,Maris coaching,NaN,Maris coaching,...,1,0,1,Maris coaching,N,Coaching. Relatie coaching. Echtscheidingsmedi...,2016,8,1,0
346,12057660,15900622,B.V. Baekerhoes,Kerkstraat 1,6051 JG MAASBRACHT,Heuvelstraat 8,6051 KT MAASBRACHT,B.V. Baekerhoes,NaN,B.V. Baekerhoes,...,2,1,2,B.V. Baekerhoes,N,Het beheer en de exploitatie van gemeenschaps ...,2005,4,28,0
347,52171396,22128468,Lukt Beheer B.V.,Palestrinalaan 12,3723 KN BILTHOVEN,Palestrinalaan 12,3723 KN BILTHOVEN,Lukt Beheer B.V.,NaN,Lukt Beheer B.V.,...,0,0,0,Lukt Beheer B.V.,N,Het beheren van stamrechtgelden alsmede financ...,2011,3,1,0
348,37139225,21512620,Stichting Valere Bewind,Oostzijde 302,1508 ET ZAANDAM,Postbus 1035,1500 AA ZAANDAM,Stichting Valere Bewind,NaN,Stichting Valere Bewind,...,0,0,0,Stichting Valere Bewind,N,Bewindvoering en inkomensbeheer,2013,11,14,0


In [173]:
df_KVK2023

,DOSSIER,VGNUMMER,HN1X30,STRVA,PCPLVA,STRCA,PCPLCA,HN1X2X30,HN2X2X30,HN45,...,CD_EC_ACT,RECHTSVORM,DOMEIN,VENN_NM_DM,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label
0,20145409,11096187,De Regenboog,Vuchtstraat 174,4816 BR BREDA,Vuchtstraat 174,4816 BR BREDA,De Regenboog -gespecialiseerde,residentiële zorg-,De Regenboog -gespecial. residentiële zorg-,...,1,1,NaN,NaN,De Regenboog -gespecialiseerde residentiële zorg-,Het geven van ondersteunende begeleiding in de...,2008,10,16,0
1,41087774,20934335,Dorpscentrum 'd' Ouw School',Oude Kerkstraat 18,5507 LC VELDHOVEN,Oude Kerkstraat 18,5507 LC VELDHOVEN,Dorpscentrum 'd' Ouw School',NaN,Dorpscentrum 'd' Ouw School',...,1,74,NaN,Stichting Dorpscentrum Oerle,Dorpscentrum 'd' Ouw School',Lokaal/ruimte voor alle plaatselijke verenigin...,2010,8,3,0
2,72170980,40296318,R.K. zorg en begeleiding,Oosterhesselenstraat 187,2545 SR 'S-GRAVENHAGE,Oosterhesselenstraat 187,2545 SR 'S-GRAVENHAGE,R.K. zorg en begeleiding,NaN,R.K. zorg en begeleiding,...,0,1,NaN,NaN,R.K. zorg en begeleiding,Jeugdzorg alsmede het ondersteunen en begeleid...,2018,7,18,0
3,10030766,16862260,Cannock Chase Public,Van Heemstraweg 58,6651 KH DRUTEN,Postbus 103,6650 AC DRUTEN,Cannock Chase Public,NaN,Cannock Chase Public,...,1,41,NaN,Mandaat B.V.,Cannock Chase Public,"Het aanbieden van diensten aan overheden, geri...",1987,3,26,0
4,40397142,22583491,Sociëteit Tyche,Oude Delft 123,2611 BE DELFT,Oude Delft 123,2611 BE DELFT,Sociëteit Tyche,NaN,Sociëteit Tyche,...,0,71,www.delftschestudentenbond.nl,Delftsche Studenten Bond,Sociëteit Tyche,studentenvereniging,2011,5,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,5021448,2456540,Stg. Streekcentrum Scheerwolde,Scheerwolderweg 26,8371 WJ SCHEERWOLDE,Scheerwolderweg 26,8371 WJ SCHEERWOLDE,Stichting Streekcentrum,Scheerwolde,Stichting Streekcentrum Scheerwolde,...,1,74,NaN,Stichting Streekcentrum Scheerwolde,Stichting Streekcentrum Scheerwolde,"Het oprichten, instandhouden en exploiteren va...",1966,3,4,1
346,41261329,22226338,Cultuur op de Campus,Heyendaalsewe 141-Gymnasionkam,6525 AJ NIJMEGEN,Heyendaalsewe 141-Gymnasionkam,6525 AJ NIJMEGEN,Cultuur op de Campus,NaN,Cultuur op de Campus,...,0,71,www.ru.nl/cultuuropdecampus,Cultuur op de Campus,Cultuur op de Campus,Het bevorderen ban een levendig cultureel klim...,2011,3,16,0
347,75835843,42501059,Enjoy Dagbesteding & Zorg,Oude Twentseweg 54,8153 RC LEMELERVELD,Oude Twentseweg 54,8153 RC LEMELERVELD,Enjoy Dagbesteding & Zorg,NaN,Enjoy Dagbesteding & Zorg,...,1,11,NaN,Enjoy Dagbesteding & Zorg,Enjoy Dagbesteding & Zorg,Aanbieden van dagbesteding en zorgbesteding.,2019,9,16,0
348,41012628,23166819,Ontmoetingscentrum de Viskenij,Kosterijweg 10-A,9922 PK WESTEREMDEN,Kosterijweg 10-A,9922 PK WESTEREMDEN,Ontmoetingscentrum de Viskenij,NaN,Ontmoetingscentrum de Viskenij,...,1,74,NaN,Stichting Ontmoetingscentrum de Viskenij,Ontmoetingscentrum de Viskenij,Ontmoetingscentrum (voor verenigingen en jeugd...,2011,8,4,1


## Cleaning the data

### Cleaning the labelled data (data_sbi_select)

In [174]:
#cleaning the data
df_label = df_label.drop(['Unnamed: 0','ZGG'], axis=1) #remove first and ZGG column since they have no use
df_label.drop(df_label.tail(1).index, inplace = True) #remove last row since it's mostly empty and only contains a sum

In [175]:
#remove ? from ZZG DEF
df_label = df_label.drop(df_label[df_label['ZZG DEF'] == '?'].index)

#remove NAN from ZZG DEF
df_label.dropna(subset=["ZZG DEF"], inplace=True)

In [176]:
#check if data cleaning the ZZG DEF column worked
df_label["ZZG DEF"].value_counts()

ZZG DEF
0     440
1      56
56      1
Name: count, dtype: int64

In [177]:
#rename 'ZZG DEF' column to 'label'
df_label.rename(columns = {'ZZG DEF':'label'}, inplace = True)

In [178]:
#move label column to the end of the df
columns = list(df_label.columns) #get list of columns

columns.remove('label')  #remove 'label' from its current position
columns.append('label')  #append 'label' to the end

df_label = df_label[columns] #reorder df column

In [179]:
#print cleaned df
df_label

,RGL,DOSSIER,VGNUMMER,HN1X30,STRVA,PCPLVA,STRCA,PCPLCA,HN1X2X30,HN2X2X30,...,DAT_VOORTZ,URL,P_W_FULLT,W_P_TOTAAL,W_P_PARTT,WP_TOT_OND,H_NAAM_VOL,IND_OPHEFF,Bedrijfsomschrijving,label
3,H,57464103.0,26906627.0,Maatsch. Beheer MFC Tiendeveen,Molenweg 4a,7936 PB TIENDEVEEN,Molenweg 4a,7936 PB TIENDEVEEN,Maatschap Beheer MFC,Tiendeveen,...,0.0,NaN,0.0,0.0,0.0,0.0,Maatschap Beheer MFC Tiendeveen,N,Het beheren van het Multifunctioneel Centrum i...,1
4,H,40446804.0,22033890.0,Aktiviteitencentrum Alletahof,Aletta Jacobsplantsoen 97,2253 RP VOORSCHOTEN,Noortheystraat 24,2251 SX VOORSCHOTEN,Aktiviteitencentrum Alletahof,NaN,...,0.0,www.alettahof.nl,0.0,0.0,0.0,0.0,Aktiviteitencentrum Alletahof,N,Buurt- en clubhuiswerk en samenlevingsopbouwwerk,1
5,H,62140892.0,31178138.0,Coöperat. Stat.park Deurne UA,Dagpauwoog 11,5754 DN DEURNE,Dagpauwoog 11,5754 DN DEURNE,Coöperatie Stationspark,Deurne UA,...,0.0,NaN,0.0,0.0,0.0,0.0,Coöperatie Stationspark Deurne U,N,"De kwaliteit van de woon-, werk- en leefomgevi...",1
6,H,86879405.0,52820130.0,Coöperatie Sustar UA,Gunninglaan 71,7314 NE APELDOORN,Gunninglaan 71,7314 NE APELDOORN,Coöperatie Sustar UA,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,Coöperatie Sustar UA,N,Het bieden van ondersteuning aan organisaties ...,1
7,H,40047714.0,22482598.0,"Jasmijnsteede, Centraal wonen",Jasmijnlaan 27,9301 NW RODEN,Jasmijnlaan 27,9301 NW RODEN,"Jasmijnsteede, Centraal wonen",voor ouderen te Roden,...,0.0,NaN,0.0,0.0,0.0,0.0,"Jasmijnsteede, Centraal wonen vo",N,Groepswonen voor ouderen.,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,H,62086715.0,31131778.0,Stichting Jalta,Keizersgracht 794,1017 EC AMSTERDAM,Keizersgracht 794,1017 EC AMSTERDAM,Stichting Jalta,NaN,...,0.0,www.stichtingjalta.nl,0.0,0.0,0.0,0.0,Stichting Jalta,N,Het initiëren en financieel steunen van initia...,0
496,H,41038939.0,22595953.0,Stichting De Zandkreek,Koeweg 17,8051 PM HATTEM,Postbus 757,3300 AT DORDRECHT,Stichting De Zandkreek,NaN,...,0.0,www.nationaalkamp.nl,0.0,0.0,0.0,0.0,Stichting De Zandkreek,N,"Het faciliteren van stichtingen, verenigingen ...",0
497,H,30225903.0,21840016.0,St.Noodfonds Vwg Utr.Heuvelrug,Kerkplein 18,3941 HV DOORN,Kerkplein 18,3941 HV DOORN,St.Noodfonds Vluchtel.werkgr.,Utr.Heuvelrug,...,0.0,NaN,0.0,0.0,0.0,0.0,St.Noodfonds Vluchtelingwerkgr.,N,Het financieel ondersteunen van vluchtelingen ...,0
498,H,52474798.0,22398171.0,Stichting Collaboration O,Daalakkersweg 6-56,5641 JA EINDHOVEN,Daalakkersweg 6-56,5641 JA EINDHOVEN,Stichting Collaboration O,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,Stichting Collaboration O,N,Zich inzetten om studerende en startende onder...,0


### Cleaning the monitor data (Resultaat)

In [180]:
#add column 'label' with all value 1 since they are all caring communities
df_monitor['label'] = 1

In [181]:
#remove columns 'DAT_ONTB_RP' and 'DAT_UITSCH RP' since they only contain 0's
df_monitor = df_monitor.drop(['DAT_ONTB_RP','DAT_UITSCH RP'], axis=1) 

In [182]:
df_monitor

,RGL,DOSSIER,VGNUMMER,HN1X30,STRVA,PCPLVA,STRCA,PCPLCA,HN1X2X30,HN2X2X30,...,DAT_VOORTZ,P_W_FULLT,W_P_TOTAAL,W_P_PARTT,WP_TOT_OND,IND_OPHEFF,H_NAAM_VOL,VENN_NM_DM,Bedrijfsomschrijving,label
0,H,2083641,22592512.0,De Schutse,Wieger van den Bosstraat 13-A,9901 GS APPINGEDAM,Wieger van den Bosstraat 13-A,9901 GS APPINGEDAM,De Schutse,NaN,...,0,0,0,0,0,N,De Schutse,Stichting voor Vrijwillige Palliatieve Termina...,Het zelfstandig dan wel in samenwerking met an...,1
1,H,4054894,NaN,Ver. Dorpsgemeenschap Havelte,Parkkamp 37,7971 AL HAVELTE,Parkkamp 37,7971 AL HAVELTE,Vereniging Dorpsgemeenschap,Havelte,...,0,0,1,1,1,N,Vereniging Dorpsgemeenschap Have,Vereniging Dorpsgemeenschap Havelte,Buurt- en clubhuiswerk en samenlevingsopbouwwe...,1
2,H,4055518,NaN,Dorpsbelang Vledder,Vledderweg 17,8381 AB VLEDDER,Vledderweg 17,8381 AB VLEDDER,Dorpsbelang Vledder,NaN,...,0,0,0,0,0,N,Dorpsbelang Vledder,Dorpsbelang Vledder,Buurt- en clubhuiswerk en samenlevingsopbouwwe...,1
3,H,8139803,22428755.0,ZorgSaam,Zwolseweg 290,7315 GZ APELDOORN,Zwolseweg 290,7315 GZ APELDOORN,ZorgSaam,NaN,...,0,0,0,0,0,N,ZorgSaam,Stichting Beheer AWBZ-gebonden vermogen ZorgSa...,Het beheren en administreren van gelden en and...,1
4,H,8186389,NaN,Stichting MFA de Zweede,Boekelosestraat 275,7548 AS ENSCHEDE,Postbus 10014,7504 PA ENSCHEDE,Stichting MFA de Zweede,NaN,...,0,0,0,0,0,N,Stichting MFA de Zweede,Stichting MFA de Zweede,"Het ontwikkelen, (doen) realiseren, financiere...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,H,81234198,NaN,Stg. Buurtfabr. Altev./Cranev.,Peter van Anrooylaan 1,6815 GT ARNHEM,Mozartstraat 15,6815 CS ARNHEM,Stichting,Buurtfabr. Alteveer/Cranevelt,...,0,0,0,0,0,N,Stichting Buurtfabriek Alteveer/,Stichting Buurtfabriek Alteveer/Cranevelt,"Bijdragen aan een sociaal krachtige buurt, Alt...",1
444,H,83748938,NaN,Stichting Lang Leve Kolham,Rengerslaan 40,9615 AM KOLHAM,Rengerslaan 40,9615 AM KOLHAM,Stichting Lang Leve Kolham,NaN,...,0,0,0,0,0,N,Stichting Lang Leve Kolham,Stichting Lang Leve Kolham,A. het realiseren van een woonvorm voor oudere...,1
445,H,84410744,NaN,Ver. Gasteren Goed voor Elkaar,Oudemolenseweg 8,9466 PB GASTEREN,Oudemolenseweg 8,9466 PB GASTEREN,Vereniging,Gasteren Goed voor Elkaar,...,0,0,0,0,0,N,Vereniging Gasteren Goed voor El,Vereniging Gasteren Goed voor Elkaar,Het in Gasteren en omstreken aanbieden van fac...,1
446,H,86130757,52124886.0,Zorgcoöperatie Nederland UA,Randwycksingel 20-A 02,6229 EE MAASTRICHT,Randwycksingel 20-A 02,6229 EE MAASTRICHT,Zorgcoöperatie Nederland UA,NaN,...,0,1,1,0,1,N,Zorgcoöperatie Nederland UA,Zorgcoöperatie Nederland UA,Het faciliteren dat verschillende zorgaanbiede...,1


### Merging the labelled datasets

In [183]:
#get column names of each df
columns_df1 = set(df_label.columns)
columns_df2 = set(df_monitor.columns)

#find common columns
common_columns = columns_df1.intersection(columns_df2)

#find columns unique to each df
unique_columns_df1 = columns_df1 - common_columns
unique_columns_df2 = columns_df2 - common_columns

print("Common Columns:", common_columns)
print("Unique Columns in df_label:", unique_columns_df1)
print("Unique Columns in df_monitor:", unique_columns_df2)

Common Columns: {'RSIN', 'H_NAAM_VOL', 'DAT_OPRI_A', 'INSCHR_DAT', 'UITS_REDEN', 'label', 'WP_TOT_OND', 'GEMNAAM', 'URL', 'W_P_PARTT', 'INS_REDEN', 'STRVA', 'ACHTERNAAM', 'Bedrijfsomschrijving', 'BOEKJAAR', 'NEVENACT_1', 'DAT_VOORTZ', 'HN1X30', 'PROV', 'VOORVOEGSE', 'OPHEFF_DAT', 'DAT_OPRICH', 'HN2X2X30', 'VENN_NM_DM', 'NEVENACT_2', 'DAT_DEP_JS', 'PCPLCA', 'GEMK_VA', 'GEMK_CA', 'PCPLVA', 'REDEN_OPH', 'RECHTSVORM', 'SBI_CODE', 'IND_OPHEFF', 'KL_WP_TOT', 'RGL', 'VOORLETTER', 'PEILDAT_WP', 'TEL_NRS', 'VGNUMMER', 'DAT_VEST', 'P_W_FULLT', 'FUNCTIE', 'SBI_OMSCHR', 'HN45', 'HFD_N_VEST', 'HN1X2X30', 'KL_WP_FULL', 'DOSSIER', 'VEST_DATUM', 'MOB_TEL_NR', 'NMI', 'STRCA', 'CD_EC_ACT', 'W_P_TOTAAL'}
Unique Columns in df_label: {'P_DAT_WP_O'}
Unique Columns in df_monitor: {'PEILDAT_WP_OND'}


In [184]:
#rename 'P_DAT_WP_O' column to 'PEILDAT_WP_OND' in df_label so that it matches with the column name in df_monitor
df_label.rename(columns = {'P_DAT_WP_O':'PEILDAT_WP_OND'}, inplace = True)

In [185]:
#merging the dataframes
df_labelled = pd.concat([df_label, df_monitor])

In [186]:
df_labelled

,RGL,DOSSIER,VGNUMMER,HN1X30,STRVA,PCPLVA,STRCA,PCPLCA,HN1X2X30,HN2X2X30,...,DAT_VOORTZ,URL,P_W_FULLT,W_P_TOTAAL,W_P_PARTT,WP_TOT_OND,H_NAAM_VOL,IND_OPHEFF,Bedrijfsomschrijving,label
3,H,57464103.0,26906627.0,Maatsch. Beheer MFC Tiendeveen,Molenweg 4a,7936 PB TIENDEVEEN,Molenweg 4a,7936 PB TIENDEVEEN,Maatschap Beheer MFC,Tiendeveen,...,0.0,NaN,0.0,0.0,0.0,0.0,Maatschap Beheer MFC Tiendeveen,N,Het beheren van het Multifunctioneel Centrum i...,1
4,H,40446804.0,22033890.0,Aktiviteitencentrum Alletahof,Aletta Jacobsplantsoen 97,2253 RP VOORSCHOTEN,Noortheystraat 24,2251 SX VOORSCHOTEN,Aktiviteitencentrum Alletahof,NaN,...,0.0,www.alettahof.nl,0.0,0.0,0.0,0.0,Aktiviteitencentrum Alletahof,N,Buurt- en clubhuiswerk en samenlevingsopbouwwerk,1
5,H,62140892.0,31178138.0,Coöperat. Stat.park Deurne UA,Dagpauwoog 11,5754 DN DEURNE,Dagpauwoog 11,5754 DN DEURNE,Coöperatie Stationspark,Deurne UA,...,0.0,NaN,0.0,0.0,0.0,0.0,Coöperatie Stationspark Deurne U,N,"De kwaliteit van de woon-, werk- en leefomgevi...",1
6,H,86879405.0,52820130.0,Coöperatie Sustar UA,Gunninglaan 71,7314 NE APELDOORN,Gunninglaan 71,7314 NE APELDOORN,Coöperatie Sustar UA,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,Coöperatie Sustar UA,N,Het bieden van ondersteuning aan organisaties ...,1
7,H,40047714.0,22482598.0,"Jasmijnsteede, Centraal wonen",Jasmijnlaan 27,9301 NW RODEN,Jasmijnlaan 27,9301 NW RODEN,"Jasmijnsteede, Centraal wonen",voor ouderen te Roden,...,0.0,NaN,0.0,0.0,0.0,0.0,"Jasmijnsteede, Centraal wonen vo",N,Groepswonen voor ouderen.,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,H,81234198.0,NaN,Stg. Buurtfabr. Altev./Cranev.,Peter van Anrooylaan 1,6815 GT ARNHEM,Mozartstraat 15,6815 CS ARNHEM,Stichting,Buurtfabr. Alteveer/Cranevelt,...,0.0,www.onzebuurtfabriek.nl,0.0,0.0,0.0,0.0,Stichting Buurtfabriek Alteveer/,N,"Bijdragen aan een sociaal krachtige buurt, Alt...",1
444,H,83748938.0,NaN,Stichting Lang Leve Kolham,Rengerslaan 40,9615 AM KOLHAM,Rengerslaan 40,9615 AM KOLHAM,Stichting Lang Leve Kolham,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,Stichting Lang Leve Kolham,N,A. het realiseren van een woonvorm voor oudere...,1
445,H,84410744.0,NaN,Ver. Gasteren Goed voor Elkaar,Oudemolenseweg 8,9466 PB GASTEREN,Oudemolenseweg 8,9466 PB GASTEREN,Vereniging,Gasteren Goed voor Elkaar,...,0.0,NaN,0.0,0.0,0.0,0.0,Vereniging Gasteren Goed voor El,N,Het in Gasteren en omstreken aanbieden van fac...,1
446,H,86130757.0,52124886.0,Zorgcoöperatie Nederland UA,Randwycksingel 20-A 02,6229 EE MAASTRICHT,Randwycksingel 20-A 02,6229 EE MAASTRICHT,Zorgcoöperatie Nederland UA,NaN,...,0.0,NaN,1.0,1.0,0.0,1.0,Zorgcoöperatie Nederland UA,N,Het faciliteren dat verschillende zorgaanbiede...,1


### Cleaning the merged labelled dataset

In [187]:
#stichting BOOR is a single organisation that exists of multiple schools/education iniatives. It is not a caring initiative
df_labelled = df_labelled.drop(df_labelled[df_labelled['VENN_NM_DM'] == 'Stichting BOOR'].index)
df_labelled

,RGL,DOSSIER,VGNUMMER,HN1X30,STRVA,PCPLVA,STRCA,PCPLCA,HN1X2X30,HN2X2X30,...,DAT_VOORTZ,URL,P_W_FULLT,W_P_TOTAAL,W_P_PARTT,WP_TOT_OND,H_NAAM_VOL,IND_OPHEFF,Bedrijfsomschrijving,label
3,H,57464103.0,26906627.0,Maatsch. Beheer MFC Tiendeveen,Molenweg 4a,7936 PB TIENDEVEEN,Molenweg 4a,7936 PB TIENDEVEEN,Maatschap Beheer MFC,Tiendeveen,...,0.0,NaN,0.0,0.0,0.0,0.0,Maatschap Beheer MFC Tiendeveen,N,Het beheren van het Multifunctioneel Centrum i...,1
4,H,40446804.0,22033890.0,Aktiviteitencentrum Alletahof,Aletta Jacobsplantsoen 97,2253 RP VOORSCHOTEN,Noortheystraat 24,2251 SX VOORSCHOTEN,Aktiviteitencentrum Alletahof,NaN,...,0.0,www.alettahof.nl,0.0,0.0,0.0,0.0,Aktiviteitencentrum Alletahof,N,Buurt- en clubhuiswerk en samenlevingsopbouwwerk,1
5,H,62140892.0,31178138.0,Coöperat. Stat.park Deurne UA,Dagpauwoog 11,5754 DN DEURNE,Dagpauwoog 11,5754 DN DEURNE,Coöperatie Stationspark,Deurne UA,...,0.0,NaN,0.0,0.0,0.0,0.0,Coöperatie Stationspark Deurne U,N,"De kwaliteit van de woon-, werk- en leefomgevi...",1
6,H,86879405.0,52820130.0,Coöperatie Sustar UA,Gunninglaan 71,7314 NE APELDOORN,Gunninglaan 71,7314 NE APELDOORN,Coöperatie Sustar UA,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,Coöperatie Sustar UA,N,Het bieden van ondersteuning aan organisaties ...,1
7,H,40047714.0,22482598.0,"Jasmijnsteede, Centraal wonen",Jasmijnlaan 27,9301 NW RODEN,Jasmijnlaan 27,9301 NW RODEN,"Jasmijnsteede, Centraal wonen",voor ouderen te Roden,...,0.0,NaN,0.0,0.0,0.0,0.0,"Jasmijnsteede, Centraal wonen vo",N,Groepswonen voor ouderen.,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,H,81234198.0,NaN,Stg. Buurtfabr. Altev./Cranev.,Peter van Anrooylaan 1,6815 GT ARNHEM,Mozartstraat 15,6815 CS ARNHEM,Stichting,Buurtfabr. Alteveer/Cranevelt,...,0.0,www.onzebuurtfabriek.nl,0.0,0.0,0.0,0.0,Stichting Buurtfabriek Alteveer/,N,"Bijdragen aan een sociaal krachtige buurt, Alt...",1
444,H,83748938.0,NaN,Stichting Lang Leve Kolham,Rengerslaan 40,9615 AM KOLHAM,Rengerslaan 40,9615 AM KOLHAM,Stichting Lang Leve Kolham,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,Stichting Lang Leve Kolham,N,A. het realiseren van een woonvorm voor oudere...,1
445,H,84410744.0,NaN,Ver. Gasteren Goed voor Elkaar,Oudemolenseweg 8,9466 PB GASTEREN,Oudemolenseweg 8,9466 PB GASTEREN,Vereniging,Gasteren Goed voor Elkaar,...,0.0,NaN,0.0,0.0,0.0,0.0,Vereniging Gasteren Goed voor El,N,Het in Gasteren en omstreken aanbieden van fac...,1
446,H,86130757.0,52124886.0,Zorgcoöperatie Nederland UA,Randwycksingel 20-A 02,6229 EE MAASTRICHT,Randwycksingel 20-A 02,6229 EE MAASTRICHT,Zorgcoöperatie Nederland UA,NaN,...,0.0,NaN,1.0,1.0,0.0,1.0,Zorgcoöperatie Nederland UA,N,Het faciliteren dat verschillende zorgaanbiede...,1


#### Generating VGNUMMER values for the rows that have nan values

In [188]:
#as the duplicates will be filtered based on VGNUMMER, it will first be checked if there are any NaN values
missing_vgnummer_rows = df_labelled[df_labelled['VGNUMMER'].isnull()].copy()
missing_vgnummer_rows

,RGL,DOSSIER,VGNUMMER,HN1X30,STRVA,PCPLVA,STRCA,PCPLCA,HN1X2X30,HN2X2X30,...,DAT_VOORTZ,URL,P_W_FULLT,W_P_TOTAAL,W_P_PARTT,WP_TOT_OND,H_NAAM_VOL,IND_OPHEFF,Bedrijfsomschrijving,label
500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56
1,H,4054894.0,NaN,Ver. Dorpsgemeenschap Havelte,Parkkamp 37,7971 AL HAVELTE,Parkkamp 37,7971 AL HAVELTE,Vereniging Dorpsgemeenschap,Havelte,...,0.0,www.dorpsgemeenschaphavelte.nl,0.0,1.0,1.0,1.0,Vereniging Dorpsgemeenschap Have,N,Buurt- en clubhuiswerk en samenlevingsopbouwwe...,1
2,H,4055518.0,NaN,Dorpsbelang Vledder,Vledderweg 17,8381 AB VLEDDER,Vledderweg 17,8381 AB VLEDDER,Dorpsbelang Vledder,NaN,...,0.0,www.dorpsbelangvledder.nl,0.0,0.0,0.0,0.0,Dorpsbelang Vledder,N,Buurt- en clubhuiswerk en samenlevingsopbouwwe...,1
4,H,8186389.0,NaN,Stichting MFA de Zweede,Boekelosestraat 275,7548 AS ENSCHEDE,Postbus 10014,7504 PA ENSCHEDE,Stichting MFA de Zweede,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,Stichting MFA de Zweede,N,"Het ontwikkelen, (doen) realiseren, financiere...",1
5,H,8209460.0,NaN,Stichting Present Kampen,Burgwal 45,8261 EP KAMPEN,Burgwal 45,8261 EP KAMPEN,Stichting Present Kampen,NaN,...,0.0,www.stichtingpresent-kampen.nl,0.0,0.0,0.0,0.0,Stichting Present Kampen,N,Vanuit een christelijke sociale basis een maat...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,H,81099940.0,NaN,Prot. gem. Geloven in Spangen,Jan Luykenstraat 12,3027 EC ROTTERDAM,Jan Luykenstraat 12,3027 EC ROTTERDAM,Protestantse gemeente,'Geloven in Spangen' te R'dam',...,0.0,NaN,0.0,0.0,0.0,0.0,Protestantse gemeente 'Geloven i,N,Religieuze organisatie,1
443,H,81234198.0,NaN,Stg. Buurtfabr. Altev./Cranev.,Peter van Anrooylaan 1,6815 GT ARNHEM,Mozartstraat 15,6815 CS ARNHEM,Stichting,Buurtfabr. Alteveer/Cranevelt,...,0.0,www.onzebuurtfabriek.nl,0.0,0.0,0.0,0.0,Stichting Buurtfabriek Alteveer/,N,"Bijdragen aan een sociaal krachtige buurt, Alt...",1
444,H,83748938.0,NaN,Stichting Lang Leve Kolham,Rengerslaan 40,9615 AM KOLHAM,Rengerslaan 40,9615 AM KOLHAM,Stichting Lang Leve Kolham,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,Stichting Lang Leve Kolham,N,A. het realiseren van een woonvorm voor oudere...,1
445,H,84410744.0,NaN,Ver. Gasteren Goed voor Elkaar,Oudemolenseweg 8,9466 PB GASTEREN,Oudemolenseweg 8,9466 PB GASTEREN,Vereniging,Gasteren Goed voor Elkaar,...,0.0,NaN,0.0,0.0,0.0,0.0,Vereniging Gasteren Goed voor El,N,Het in Gasteren en omstreken aanbieden van fac...,1


In [189]:
#check if there are any duplicates within the missing rows
duplicates_in_missing_vgnummer = missing_vgnummer_rows[missing_vgnummer_rows.duplicated(subset='HN45', keep=False)]

#display the duplicate rows
print(duplicates_in_missing_vgnummer)

Empty DataFrame
Columns: [RGL, DOSSIER, VGNUMMER, HN1X30, STRVA, PCPLVA, STRCA, PCPLCA, HN1X2X30, HN2X2X30, HN45, GEMK_VA, GEMK_CA, GEMNAAM, PROV, TEL_NRS, FUNCTIE, VOORLETTER, VOORVOEGSE, ACHTERNAAM, SBI_CODE, SBI_OMSCHR, NEVENACT_1, NEVENACT_2, HFD_N_VEST, CD_EC_ACT, KL_WP_TOT, KL_WP_FULL, MOB_TEL_NR, PEILDAT_WP, PEILDAT_WP_OND, RECHTSVORM, INS_REDEN, UITS_REDEN, REDEN_OPH, RSIN, VENN_NM_DM, NMI, BOEKJAAR, DAT_OPRI_A, DAT_DEP_JS, INSCHR_DAT, OPHEFF_DAT, DAT_OPRICH, DAT_VEST, VEST_DATUM, DAT_VOORTZ, URL, P_W_FULLT, W_P_TOTAAL, W_P_PARTT, WP_TOT_OND, H_NAAM_VOL, IND_OPHEFF, Bedrijfsomschrijving, label]
Index: []

[0 rows x 56 columns]


There are no duplicates within the missing rows.

In [190]:
#check if there are any rows in the 2022 data that do include the VGNUMMER that is missing in the labelled data
occurs_in_KVK2022 = missing_vgnummer_rows['DOSSIER'].isin(df_KVK2022['DOSSIER'])

#filter missing_vgnummer_rows to get the subset that occurs in df_KVK2022
missing_vgnummer_in_KVK2022 = missing_vgnummer_rows[occurs_in_KVK2022]
missing_vgnummer_in_KVK2022 

,RGL,DOSSIER,VGNUMMER,HN1X30,STRVA,PCPLVA,STRCA,PCPLCA,HN1X2X30,HN2X2X30,...,DAT_VOORTZ,URL,P_W_FULLT,W_P_TOTAAL,W_P_PARTT,WP_TOT_OND,H_NAAM_VOL,IND_OPHEFF,Bedrijfsomschrijving,label


There are no rows in the 2022 dataset that can help fill in the missing values in the labelled dataset.

In [191]:
#check if there are any rows in the 2023 data that do include the VGNUMMER that is missing in the labelled data
occurs_in_KVK2023 = missing_vgnummer_rows['DOSSIER'].isin(df_KVK2023['DOSSIER'])

#filter missing_vgnummer_rows to get the subset that occurs in df_KVK2022
missing_vgnummer_in_KVK2023 = missing_vgnummer_rows[occurs_in_KVK2023]
missing_vgnummer_in_KVK2023

,RGL,DOSSIER,VGNUMMER,HN1X30,STRVA,PCPLVA,STRCA,PCPLCA,HN1X2X30,HN2X2X30,...,DAT_VOORTZ,URL,P_W_FULLT,W_P_TOTAAL,W_P_PARTT,WP_TOT_OND,H_NAAM_VOL,IND_OPHEFF,Bedrijfsomschrijving,label


In the 2023 dataset, there are rows that are also present in the labelled dataset with missing VGNUMMER, but unfortunatly they also do not have a VGNUMMER.

Since there is no data in the 2022 and 2023 datasets that can help fill in the missing values for VGNUMMER, the values will be randomly generated. By doing this we can still utilize these rows and don't have to delete them.

In [192]:
#function to generate unique VGNUMMER values for missing values
def generate_unique_vgnummers_for_missing(df):
    missing_vgnummers = df[df['VGNUMMER'].isnull()]
    existing_vgnummers = set(df['VGNUMMER'].dropna())
    new_vgnummers = set(range(10000, 10000 + len(existing_vgnummers) + len(missing_vgnummers)))
    unique_vgnummers = new_vgnummers - existing_vgnummers
    return list(unique_vgnummers)

#get the unique VGNUMMER values for missing values
unique_vgnummers_for_missing = generate_unique_vgnummers_for_missing(df_labelled)

#update VGNUMMER column for missing values with unique values
missing_vgnummer_rows = df_labelled[df_labelled['VGNUMMER'].isnull()].copy()
missing_vgnummer_rows['VGNUMMER'] = unique_vgnummers_for_missing[:len(missing_vgnummer_rows)]

#concatenate the original dataframe with the updated missing values dataframe
df_labelled_updated = pd.concat([df_labelled[df_labelled['VGNUMMER'].notnull()], missing_vgnummer_rows])

#assign the updated dataframe back to df_labelled
df_labelled = df_labelled_updated

#### Removing duplicates

The duplicates are removed using VGNUMMER as the unique identifier.

In [193]:
#check for duplicates using VGNUMMER as unique identifier
duplicate_rows = df_labelled[df_labelled.duplicated(subset=['VGNUMMER'], keep=False)]

if not duplicate_rows.empty:
    print("Duplicate rows based on VGNUMMER:")
    print(duplicate_rows)
else:
    print("No duplicate rows found.")

Duplicate rows based on VGNUMMER:
    RGL     DOSSIER    VGNUMMER                          HN1X30  \
16    H  61801070.0  30871883.0  Doel Coöp. Samen E.(k)b. II UA   
440   H  61801070.0  30871883.0  Doel Coöp. Samen E.(k)b. II UA   

                        STRVA           PCPLVA                     STRCA  \
16   Kard van Rossumstraat 27  5104 HK  DONGEN  Kard van Rossumstraat 27   
440  Kard van Rossumstraat 27  5104 HK  DONGEN  Kard van Rossumstraat 27   

              PCPLCA               HN1X2X30                   HN2X2X30  ...  \
16   5104 HK  DONGEN  Doel Coöperatie Samen  Energie(k) besparen II UA  ...   
440  5104 HK  DONGEN  Doel Coöperatie Samen  Energie(k) besparen II UA  ...   

    DAT_VOORTZ  URL  P_W_FULLT W_P_TOTAAL W_P_PARTT WP_TOT_OND  \
16         0.0  NaN        0.0        1.0       1.0        1.0   
440        0.0  NaN        0.0        1.0       1.0        1.0   

                           H_NAAM_VOL IND_OPHEFF  \
16   Doel Coöperatie Samen Energie(k)         

2 identical organisations are identified.

In [194]:
#specify names of duplicate organisations to remove
instances_to_remove = ['Doel Coöperat. Samen Energie(k) Bespar. II UA']

#create condition to filter out rows with specified names
condition_to_remove = df_labelled['HN45'].isin(instances_to_remove) & ~df_labelled['HN45'].duplicated()

#apply the condition to filter the df
df_labelled = df_labelled[~condition_to_remove]

#### Removing columns that are not useful for analysis

For the reasoning behind why certain column are dropped, please refer to the report.

In [195]:
df_labelled.columns

Index(['RGL', 'DOSSIER', 'VGNUMMER', 'HN1X30', 'STRVA', 'PCPLVA', 'STRCA',
       'PCPLCA', 'HN1X2X30', 'HN2X2X30', 'HN45', 'GEMK_VA', 'GEMK_CA',
       'GEMNAAM', 'PROV', 'TEL_NRS', 'FUNCTIE', 'VOORLETTER', 'VOORVOEGSE',
       'ACHTERNAAM', 'SBI_CODE', 'SBI_OMSCHR', 'NEVENACT_1', 'NEVENACT_2',
       'HFD_N_VEST', 'CD_EC_ACT', 'KL_WP_TOT', 'KL_WP_FULL', 'MOB_TEL_NR',
       'PEILDAT_WP', 'PEILDAT_WP_OND', 'RECHTSVORM', 'INS_REDEN', 'UITS_REDEN',
       'REDEN_OPH', 'RSIN', 'VENN_NM_DM', 'NMI', 'BOEKJAAR', 'DAT_OPRI_A',
       'DAT_DEP_JS', 'INSCHR_DAT', 'OPHEFF_DAT', 'DAT_OPRICH', 'DAT_VEST',
       'VEST_DATUM', 'DAT_VOORTZ', 'URL', 'P_W_FULLT', 'W_P_TOTAAL',
       'W_P_PARTT', 'WP_TOT_OND', 'H_NAAM_VOL', 'IND_OPHEFF',
       'Bedrijfsomschrijving', 'label'],
      dtype='object')

In [196]:
columns_to_drop = ['RGL', 'DOSSIER', 'VGNUMMER', 'HN1X30', 'STRVA', 'PCPLVA', 'STRCA','PCPLCA', 'HN1X2X30', 'HN2X2X30', 
                   'GEMK_CA', 'GEMK_VA', 'GEMNAAM', 'TEL_NRS','FUNCTIE', 'VOORLETTER', 'VOORVOEGSE', 'ACHTERNAAM',
                   'SBI_OMSCHR', 'NEVENACT_1', 'NEVENACT_2','HFD_N_VEST','CD_EC_ACT', 'MOB_TEL_NR', 'PEILDAT_WP', 
                   'PEILDAT_WP_OND', 'INS_REDEN', 'UITS_REDEN', 'REDEN_OPH','RSIN', 'NMI','BOEKJAAR', 'DAT_OPRI_A', 
                   'DAT_DEP_JS', 'OPHEFF_DAT', 'DAT_VEST', 'VEST_DATUM', 'DAT_VOORTZ', 'URL', 'WP_TOT_OND', 
                   'IND_OPHEFF', 'DAT_OPRICH', 'VENN_NM_DM', 'KL_WP_FULL', 'W_P_PARTT', 'KL_WP_TOT', 'P_W_FULLT']

In [197]:
#drop the columns
df_labelled.drop(columns=columns_to_drop, inplace=True)

C:\Users\SaidKalee\AppData\Local\Temp\ipykernel_26868\3342210241.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labelled.drop(columns=columns_to_drop, inplace=True)


In [198]:
df_labelled

,HN45,PROV,SBI_CODE,RECHTSVORM,INSCHR_DAT,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,label
3,Maatschap Beheer MFC Tiendeveen,D,88999.0,7.0,20130314.0,0.0,Maatschap Beheer MFC Tiendeveen,Het beheren van het Multifunctioneel Centrum i...,1
4,Aktiviteitencentrum Alletahof,H,88993.0,71.0,20110216.0,0.0,Aktiviteitencentrum Alletahof,Buurt- en clubhuiswerk en samenlevingsopbouwwerk,1
5,Coöperatie Stationspark Deurne UA,P,94997.0,61.0,20141223.0,0.0,Coöperatie Stationspark Deurne U,"De kwaliteit van de woon-, werk- en leefomgevi...",1
6,Coöperatie Sustar UA,G,94997.0,61.0,20220701.0,0.0,Coöperatie Sustar UA,Het bieden van ondersteuning aan organisaties ...,1
7,"Jasmijnsteede, Centraal wonen voor ouderen",D,88102.0,71.0,20110420.0,0.0,"Jasmijnsteede, Centraal wonen vo",Groepswonen voor ouderen.,1
...,...,...,...,...,...,...,...,...,...
442,Prot. gem. 'Geloven in Spangen' te Rotterdam,H,94911.0,73.0,0.0,0.0,Protestantse gemeente 'Geloven i,Religieuze organisatie,1
443,Stichting Buurtfabriek Alteveer/Cranevelt,G,88993.0,74.0,0.0,0.0,Stichting Buurtfabriek Alteveer/,"Bijdragen aan een sociaal krachtige buurt, Alt...",1
444,Stichting Lang Leve Kolham,A,94996.0,74.0,0.0,0.0,Stichting Lang Leve Kolham,A. het realiseren van een woonvorm voor oudere...,1
445,Vereniging Gasteren Goed voor Elkaar,D,88993.0,71.0,0.0,0.0,Vereniging Gasteren Goed voor El,Het in Gasteren en omstreken aanbieden van fac...,1


#### Find missing data

In [199]:
#amount of missing data per column
missing_values_per_column = df_labelled.isna().sum()

print(missing_values_per_column)

HN45                    1
PROV                    1
SBI_CODE                2
RECHTSVORM              1
INSCHR_DAT              1
W_P_TOTAAL              1
H_NAAM_VOL              1
Bedrijfsomschrijving    1
label                   0
dtype: int64


In [200]:
#find the 0 values
zero_rows = df_labelled.drop('label', axis=1)[(df_labelled.drop('label', axis=1) == 0).any(axis=1)]

print(zero_rows)

                                             HN45 PROV  SBI_CODE  RECHTSVORM  \
3                 Maatschap Beheer MFC Tiendeveen    D   88999.0         7.0   
4                   Aktiviteitencentrum Alletahof    H   88993.0        71.0   
5               Coöperatie Stationspark Deurne UA    P   94997.0        61.0   
6                            Coöperatie Sustar UA    G   94997.0        61.0   
7      Jasmijnsteede, Centraal wonen voor ouderen    D   88102.0        71.0   
..                                            ...  ...       ...         ...   
442  Prot. gem. 'Geloven in Spangen' te Rotterdam    H   94911.0        73.0   
443     Stichting Buurtfabriek Alteveer/Cranevelt    G   88993.0        74.0   
444                    Stichting Lang Leve Kolham    A   94996.0        74.0   
445          Vereniging Gasteren Goed voor Elkaar    D   88993.0        71.0   
447                           Stichting de Eester    L    6832.0        74.0   

     INSCHR_DAT  W_P_TOTAAL            

It appears that the INSCHR_DAT column has several 0 values. This value is not possible and will therefore be converted to nan values and thereafter dropped.

In [201]:
#replace the missing values that are 0 with nan
df_labelled.loc[df_labelled['INSCHR_DAT'] == 0, 'INSCHR_DAT'] = np.nan

#drop rows with nan values in the 'INSCHR_DAT', 'SBI_CODE' and 'KL_WP_TOT' column
df_labelled.dropna(subset=['INSCHR_DAT', 'SBI_CODE'], inplace=True)

C:\Users\SaidKalee\AppData\Local\Temp\ipykernel_26868\4167852352.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labelled.dropna(subset=['INSCHR_DAT', 'SBI_CODE'], inplace=True)


#### Convert to appropriate datatypes

In [202]:
#convert all float columns to integers
df_labelled = df_labelled.applymap(lambda x: int(x) if isinstance(x, float) else x)
 
#print the updated df
df_labelled

C:\Users\SaidKalee\AppData\Local\Temp\ipykernel_26868\911512652.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_labelled = df_labelled.applymap(lambda x: int(x) if isinstance(x, float) else x)


,HN45,PROV,SBI_CODE,RECHTSVORM,INSCHR_DAT,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,label
3,Maatschap Beheer MFC Tiendeveen,D,88999,7,20130314,0,Maatschap Beheer MFC Tiendeveen,Het beheren van het Multifunctioneel Centrum i...,1
4,Aktiviteitencentrum Alletahof,H,88993,71,20110216,0,Aktiviteitencentrum Alletahof,Buurt- en clubhuiswerk en samenlevingsopbouwwerk,1
5,Coöperatie Stationspark Deurne UA,P,94997,61,20141223,0,Coöperatie Stationspark Deurne U,"De kwaliteit van de woon-, werk- en leefomgevi...",1
6,Coöperatie Sustar UA,G,94997,61,20220701,0,Coöperatie Sustar UA,Het bieden van ondersteuning aan organisaties ...,1
7,"Jasmijnsteede, Centraal wonen voor ouderen",D,88102,71,20110420,0,"Jasmijnsteede, Centraal wonen vo",Groepswonen voor ouderen.,1
...,...,...,...,...,...,...,...,...,...
252,Verenig. Groepsw. van Oud. 'Ouder-Landshoek',H,94991,71,19900306,0,Vereniging Groepswonen van Ouder,Bejaardenverenigingen,1
253,Bewonersvereniging Centraal Wonen 'De Banier',H,68204,71,19791026,0,Bewonersvereniging Centraal Wone,Verhuur van onroerend goed,1
257,Stichting Dorpsraad Ven-Zelderheide,K,94997,74,19810626,5,Stichting Dorpsraad Ven-Zelderhe,"Belangenverenigingen, meer bezighouden met ge-...",1
262,Stichting Wijkraad Loven-Besterd,P,88993,74,19880704,0,Stichting Wijkraad Loven-Besterd,Buurthuiswerk en samenlevingsopbouwwerk,1


In [203]:
#convert the integer values to datetime objects
df_labelled['INSCHR_DAT'] = pd.to_datetime(df_labelled['INSCHR_DAT'], format='%Y%m%d')

#extract year, month, and day into separate columns
df_labelled['INSCHR_DAT_JAAR'] = df_labelled['INSCHR_DAT'].dt.year
df_labelled['INSCHR_DAT_MAAND'] = df_labelled['INSCHR_DAT'].dt.month
df_labelled['INSCHR_DAT_DAG'] = df_labelled['INSCHR_DAT'].dt.day

In [204]:
column_name_to_drop = 'INSCHR_DAT'
df_labelled.drop(columns=[column_name_to_drop], inplace=True)

In [205]:
df_labelled

,HN45,PROV,SBI_CODE,RECHTSVORM,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,label,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG
3,Maatschap Beheer MFC Tiendeveen,D,88999,7,0,Maatschap Beheer MFC Tiendeveen,Het beheren van het Multifunctioneel Centrum i...,1,2013,3,14
4,Aktiviteitencentrum Alletahof,H,88993,71,0,Aktiviteitencentrum Alletahof,Buurt- en clubhuiswerk en samenlevingsopbouwwerk,1,2011,2,16
5,Coöperatie Stationspark Deurne UA,P,94997,61,0,Coöperatie Stationspark Deurne U,"De kwaliteit van de woon-, werk- en leefomgevi...",1,2014,12,23
6,Coöperatie Sustar UA,G,94997,61,0,Coöperatie Sustar UA,Het bieden van ondersteuning aan organisaties ...,1,2022,7,1
7,"Jasmijnsteede, Centraal wonen voor ouderen",D,88102,71,0,"Jasmijnsteede, Centraal wonen vo",Groepswonen voor ouderen.,1,2011,4,20
...,...,...,...,...,...,...,...,...,...,...,...
252,Verenig. Groepsw. van Oud. 'Ouder-Landshoek',H,94991,71,0,Vereniging Groepswonen van Ouder,Bejaardenverenigingen,1,1990,3,6
253,Bewonersvereniging Centraal Wonen 'De Banier',H,68204,71,0,Bewonersvereniging Centraal Wone,Verhuur van onroerend goed,1,1979,10,26
257,Stichting Dorpsraad Ven-Zelderheide,K,94997,74,5,Stichting Dorpsraad Ven-Zelderhe,"Belangenverenigingen, meer bezighouden met ge-...",1,1981,6,26
262,Stichting Wijkraad Loven-Besterd,P,88993,74,0,Stichting Wijkraad Loven-Besterd,Buurthuiswerk en samenlevingsopbouwwerk,1,1988,7,4


### Cleaning data 2022+2023

#### Check duplicate data 2022

In [206]:
#specify columns to check for duplicates
duplicate_cols = ['VGNUMMER']

#check for duplicates based on specified column(s)
duplicate_rows = df_KVK2022[df_KVK2022.duplicated(subset=duplicate_cols, keep=False)]

if not duplicate_rows.empty:
    print("Duplicate Rows based on specified columns:")
    print(duplicate_rows)
else:
    print("No duplicate rows found based on specified columns.")

No duplicate rows found based on specified columns.


#### Drop unnecessary columns and  if the columns are the same as with the labelled dataset (2022)

In [207]:
#drop unneccesary columns for 2022 data
columns_to_drop2022 = [ 'DOSSIER', 'VGNUMMER', 'HN1X30', 'STRVA', 'PCPLVA', 'STRCA', 'PCPLCA', 'HN1X2X30', 
                       'HN2X2X30', 'GEMK_CA', 'GEMNAAM', 'GEMK_VA', 'TEL_NRS', 'FUNCTIE', 'VOORLETTER', 'VOORVOEGSE', 
                       'ACHTERNAAM', 'SBI_OMSCHR', 'NEVENACT_1', 'NEVENACT_2', 'HFD_N_VEST', 'CD_EC_ACT', 'MOB_TEL_NR', 
                       'PEILDAT_WP', 'P_DAT_WP_O', 'INS_REDEN', 'UITS_REDEN', 'REDEN_OPH', 'RSIN', 'NMI', 'BOEKJAAR', 
                       'DAT_OPRI_A', 'DAT_DEP_JS', 'OPHEFF_DAT', 'DAT_VEST','DAT_OPRICH' ,'VEST_DATUM', 'DAT_VOORTZ',
                       'W_P_PARTT', 'URL', 'KL_WP_FULL', 'WP_TOT_OND', 'IND_OPHEFF','VENN_NM_DM', 'KL_WP_TOT', 'P_W_FULLT']

#drop the columns
df_KVK2022.drop(columns=columns_to_drop2022, inplace=True)

In [208]:
#get column names of each df
columns_df_2022 = set(df_KVK2022.columns)
columns_df_lbl = set(df_labelled.columns)

#find common columns
common_columns2 = columns_df_2022.intersection(columns_df_lbl)

#find columns unique to each df
unique_columns_df_2022 = columns_df_2022 - common_columns2
unique_columns_lbl = columns_df_lbl - common_columns2

print("Common Columns:", common_columns2)
print("Unique Columns in df_KVK2022:", unique_columns_df_2022)
print("Unique Columns in df_labelled:", unique_columns_lbl)

Common Columns: {'H_NAAM_VOL', 'PROV', 'HN45', 'INSCHR_DAT_DAG', 'W_P_TOTAAL', 'label', 'RECHTSVORM', 'INSCHR_DAT_JAAR', 'INSCHR_DAT_MAAND', 'Bedrijfsomschrijving', 'SBI_CODE'}
Unique Columns in df_KVK2022: set()
Unique Columns in df_labelled: set()


#### Delete duplicates in 2023 data

In [209]:
#specify columns to check for duplicates
duplicate_cols = ['VGNUMMER'] 

#check for duplicates based on specified column(s)
duplicate_rows = df_KVK2023[df_KVK2023.duplicated(subset=duplicate_cols, keep=False)]

if not duplicate_rows.empty:
    print("Duplicate Rows based on specified columns:")
    print(duplicate_rows)
else:
    print("No duplicate rows found based on specified columns.")

No duplicate rows found based on specified columns.


In [210]:
duplicate_rows
duplicate_rows.sort_values(by=['HN45'])

,DOSSIER,VGNUMMER,HN1X30,STRVA,PCPLVA,STRCA,PCPLCA,HN1X2X30,HN2X2X30,HN45,...,CD_EC_ACT,RECHTSVORM,DOMEIN,VENN_NM_DM,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label


#### Drop columns and check if the columns are the same as with the labelled dataset (2023)

In [211]:
#delete columns that are unnecesary, 2023 dataframe has significantly less amount of columns, so columns to drop are different
columns_to_drop2023 = [ 'GEMK_VA', 'GEMK_CA', 'DOSSIER', 'PCPLVA', 'WP_TOT_OND','W_P_FULLT', 'HN1X30', 'PCPLCA', 
                       'HN2X2X30', 'STRVA', 'VGNUMMER', 'SBI_OMSCHR', 'GEMNAAM', 'CD_EC_ACT', 'HN1X2X30', 'STRCA', 
                       'BEHKN', 'DOMEIN', 'KL_WP_FULL', 'PCVA_CIJF', 'PCCA_LTRS', 'PCVA_LTRS', 'PCCA_CIJF', 'GEOKN', 
                       'VENN_NM_DM', 'KL_WP_TOT']

df_KVK2023.drop(columns=columns_to_drop2023, inplace=True)


In [212]:
#check if column names match up between 2023 data and the labelled dataset
columns_df_2023 = set(df_KVK2023.columns)
columns_df_lbl = set(df_labelled.columns)

#find common columns
common_columns3 = columns_df_2023.intersection(columns_df_lbl)

#find columns unique to each df
unique_columns_df_2023 = columns_df_2023 - common_columns3
unique_columns_lbl = columns_df_lbl - common_columns3

print("Common Columns:", common_columns3)
print("Unique Columns in df_KVK2023:", unique_columns_df_2023)

print("Unique Columns in df_labelled:", unique_columns_lbl)

Common Columns: {'H_NAAM_VOL', 'PROV', 'HN45', 'INSCHR_DAT_DAG', 'W_P_TOTAAL', 'label', 'RECHTSVORM', 'INSCHR_DAT_JAAR', 'INSCHR_DAT_MAAND', 'Bedrijfsomschrijving', 'SBI_CODE'}
Unique Columns in df_KVK2023: {'NEVENACT_1', 'NEVENACT_2'}
Unique Columns in df_labelled: set()


#### Missing data 2022 and 2023

In [213]:
#amount of missing data per column
missing_values_per_column = df_KVK2023.isna().sum()

print(missing_values_per_column)

HN45                      0
PROV                      0
SBI_CODE                  0
NEVENACT_1              182
NEVENACT_2              266
W_P_TOTAAL                0
RECHTSVORM                0
H_NAAM_VOL                0
Bedrijfsomschrijving      0
INSCHR_DAT_JAAR           0
INSCHR_DAT_MAAND          0
INSCHR_DAT_DAG            0
label                     0
dtype: int64


In [214]:
#amount of missing data per column
missing_values_per_column = df_KVK2022.isna().sum()

print(missing_values_per_column)

HN45                    0
PROV                    0
SBI_CODE                0
RECHTSVORM              0
W_P_TOTAAL              0
H_NAAM_VOL              0
Bedrijfsomschrijving    0
INSCHR_DAT_JAAR         0
INSCHR_DAT_MAAND        0
INSCHR_DAT_DAG          0
label                   0
dtype: int64


In [215]:
#drop all rows with missing data (only 12 in PROV so those are dropped)
df_KVK2022 = df_KVK2022.dropna(subset=['PROV'])

In [216]:
#identify the zero values that are not supposed to have value 0 (missing data) 
zero_rows = df_KVK2022[(df_KVK2022 == 0).any(axis=1)]

zero_rows

,HN45,PROV,SBI_CODE,RECHTSVORM,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label
0,Scouting Ernst van Kempengroep,M,88993,71,0,Scouting Ernst van Kempengroep,Scoutingvereniging,2011,1,24,0
1,Coöperatie Duurzaam Gortel Niersen UA,G,94996,61,0,Coöperatie Duurzaam Gortel Niers,Het bijdragen aan de verduurzaming van de same...,2019,12,24,1
2,CVVE Winkelcentr. de Gagel UA,P,94997,61,0,Coöperatieve Vereniging van Eige,Het behartigen van de gemeenschappelijke belan...,2013,11,29,0
3,Coöperatie Nieuwe Energie UA,G,94997,61,0,Coöperatie Nieuwe Energie UA,Het voorzien in de stoffelijke behoeften van h...,2020,1,28,0
4,De Verbindingskamer,H,88999,1,1,De Verbindingskamer,Adviesburo op het gebied van welzijn en sociaa...,2022,1,12,0
...,...,...,...,...,...,...,...,...,...,...,...
345,Maris coaching,L,88999,1,1,Maris coaching,Coaching. Relatie coaching. Echtscheidingsmedi...,2016,8,1,0
346,B.V. Baekerhoes,K,88999,41,2,B.V. Baekerhoes,Het beheer en de exploitatie van gemeenschaps ...,2005,4,28,0
347,Lukt Beheer B.V.,M,88999,41,0,Lukt Beheer B.V.,Het beheren van stamrechtgelden alsmede financ...,2011,3,1,0
348,Stichting Valere Bewind,L,88999,74,0,Stichting Valere Bewind,Bewindvoering en inkomensbeheer,2013,11,14,0


In [217]:
#same for 2023
zero_rows = df_KVK2023[(df_KVK2023 == 0).any(axis=1)]

zero_rows

,HN45,PROV,SBI_CODE,NEVENACT_1,NEVENACT_2,W_P_TOTAAL,RECHTSVORM,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label
0,De Regenboog -gespecial. residentiële zorg-,P,88992,NaN,NaN,1,1,De Regenboog -gespecialiseerde residentiële zorg-,Het geven van ondersteunende begeleiding in de...,2008,10,16,0
1,Dorpscentrum 'd' Ouw School',P,88999,NaN,NaN,0,74,Dorpscentrum 'd' Ouw School',Lokaal/ruimte voor alle plaatselijke verenigin...,2010,8,3,0
2,R.K. zorg en begeleiding,H,88991,88992.0,88103.0,1,1,R.K. zorg en begeleiding,Jeugdzorg alsmede het ondersteunen en begeleid...,2018,7,18,0
3,Cannock Chase Public,G,8291,78202.0,88999.0,85,41,Cannock Chase Public,"Het aanbieden van diensten aan overheden, geri...",1987,3,26,0
4,Sociëteit Tyche,H,88993,NaN,NaN,0,71,Sociëteit Tyche,studentenvereniging,2011,5,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Stichting Streekcentrum Scheerwolde,E,88999,NaN,NaN,0,74,Stichting Streekcentrum Scheerwolde,"Het oprichten, instandhouden en exploiteren va...",1966,3,4,1
346,Cultuur op de Campus,G,94994,NaN,NaN,0,71,Cultuur op de Campus,Het bevorderen ban een levendig cultureel klim...,2011,3,16,0
347,Enjoy Dagbesteding & Zorg,E,88992,NaN,NaN,2,11,Enjoy Dagbesteding & Zorg,Aanbieden van dagbesteding en zorgbesteding.,2019,9,16,0
348,Ontmoetingscentrum de Viskenij,A,88999,NaN,NaN,0,74,Ontmoetingscentrum de Viskenij,Ontmoetingscentrum (voor verenigingen en jeugd...,2011,8,4,1


In [218]:
df_KVK2022

,HN45,PROV,SBI_CODE,RECHTSVORM,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label
0,Scouting Ernst van Kempengroep,M,88993,71,0,Scouting Ernst van Kempengroep,Scoutingvereniging,2011,1,24,0
1,Coöperatie Duurzaam Gortel Niersen UA,G,94996,61,0,Coöperatie Duurzaam Gortel Niers,Het bijdragen aan de verduurzaming van de same...,2019,12,24,1
2,CVVE Winkelcentr. de Gagel UA,P,94997,61,0,Coöperatieve Vereniging van Eige,Het behartigen van de gemeenschappelijke belan...,2013,11,29,0
3,Coöperatie Nieuwe Energie UA,G,94997,61,0,Coöperatie Nieuwe Energie UA,Het voorzien in de stoffelijke behoeften van h...,2020,1,28,0
4,De Verbindingskamer,H,88999,1,1,De Verbindingskamer,Adviesburo op het gebied van welzijn en sociaa...,2022,1,12,0
...,...,...,...,...,...,...,...,...,...,...,...
345,Maris coaching,L,88999,1,1,Maris coaching,Coaching. Relatie coaching. Echtscheidingsmedi...,2016,8,1,0
346,B.V. Baekerhoes,K,88999,41,2,B.V. Baekerhoes,Het beheer en de exploitatie van gemeenschaps ...,2005,4,28,0
347,Lukt Beheer B.V.,M,88999,41,0,Lukt Beheer B.V.,Het beheren van stamrechtgelden alsmede financ...,2011,3,1,0
348,Stichting Valere Bewind,L,88999,74,0,Stichting Valere Bewind,Bewindvoering en inkomensbeheer,2013,11,14,0


In [219]:
df_KVK2023

,HN45,PROV,SBI_CODE,NEVENACT_1,NEVENACT_2,W_P_TOTAAL,RECHTSVORM,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label
0,De Regenboog -gespecial. residentiële zorg-,P,88992,NaN,NaN,1,1,De Regenboog -gespecialiseerde residentiële zorg-,Het geven van ondersteunende begeleiding in de...,2008,10,16,0
1,Dorpscentrum 'd' Ouw School',P,88999,NaN,NaN,0,74,Dorpscentrum 'd' Ouw School',Lokaal/ruimte voor alle plaatselijke verenigin...,2010,8,3,0
2,R.K. zorg en begeleiding,H,88991,88992.0,88103.0,1,1,R.K. zorg en begeleiding,Jeugdzorg alsmede het ondersteunen en begeleid...,2018,7,18,0
3,Cannock Chase Public,G,8291,78202.0,88999.0,85,41,Cannock Chase Public,"Het aanbieden van diensten aan overheden, geri...",1987,3,26,0
4,Sociëteit Tyche,H,88993,NaN,NaN,0,71,Sociëteit Tyche,studentenvereniging,2011,5,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Stichting Streekcentrum Scheerwolde,E,88999,NaN,NaN,0,74,Stichting Streekcentrum Scheerwolde,"Het oprichten, instandhouden en exploiteren va...",1966,3,4,1
346,Cultuur op de Campus,G,94994,NaN,NaN,0,71,Cultuur op de Campus,Het bevorderen ban een levendig cultureel klim...,2011,3,16,0
347,Enjoy Dagbesteding & Zorg,E,88992,NaN,NaN,2,11,Enjoy Dagbesteding & Zorg,Aanbieden van dagbesteding en zorgbesteding.,2019,9,16,0
348,Ontmoetingscentrum de Viskenij,A,88999,NaN,NaN,0,74,Ontmoetingscentrum de Viskenij,Ontmoetingscentrum (voor verenigingen en jeugd...,2011,8,4,1


In [220]:
#replace the missing values that are 0 with nan
df_KVK2022.loc[df_KVK2022['INSCHR_DAT_JAAR'] == 0, 'INSCHR_DAT_JAAR'] = np.nan
df_KVK2022.loc[df_KVK2022['INSCHR_DAT_MAAND'] == 0, 'INSCHR_DAT_MAAND'] = np.nan
df_KVK2022.loc[df_KVK2022['INSCHR_DAT_DAG'] == 0, 'INSCHR_DAT_DAG'] = np.nan

df_KVK2022.loc[df_KVK2022['PROV'] == 0, 'PROV'] = np.nan

#drop rows with nan values in the 'INSCHR_DAT'and 'PROV' column
df_KVK2022.dropna(subset=['INSCHR_DAT_JAAR','INSCHR_DAT_MAAND','INSCHR_DAT_DAG', 'PROV'], inplace=True)

In [221]:
#replace the missing values that are 0 with nan
df_KVK2023.loc[df_KVK2023['INSCHR_DAT_JAAR'] == 0, 'INSCHR_DAT_JAAR'] = np.nan
df_KVK2023.loc[df_KVK2023['INSCHR_DAT_MAAND'] == 0, 'INSCHR_DAT_MAAND'] = np.nan
df_KVK2023.loc[df_KVK2023['INSCHR_DAT_DAG'] == 0, 'INSCHR_DAT_DAG'] = np.nan

df_KVK2023.loc[df_KVK2023['PROV'] == 0, 'PROV'] = np.nan

#drop rows with nan values in the 'INSCHR_DAT'and 'PROV' column
df_KVK2023.dropna(subset=['INSCHR_DAT_JAAR','INSCHR_DAT_MAAND','INSCHR_DAT_DAG', 'PROV'], inplace=True)

In [223]:
df_KVK2023


,HN45,PROV,SBI_CODE,NEVENACT_1,NEVENACT_2,W_P_TOTAAL,RECHTSVORM,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label
0,De Regenboog -gespecial. residentiële zorg-,P,88992,NaN,NaN,1,1,De Regenboog -gespecialiseerde residentiële zorg-,Het geven van ondersteunende begeleiding in de...,2008.0,10.0,16.0,0
1,Dorpscentrum 'd' Ouw School',P,88999,NaN,NaN,0,74,Dorpscentrum 'd' Ouw School',Lokaal/ruimte voor alle plaatselijke verenigin...,2010.0,8.0,3.0,0
2,R.K. zorg en begeleiding,H,88991,88992.0,88103.0,1,1,R.K. zorg en begeleiding,Jeugdzorg alsmede het ondersteunen en begeleid...,2018.0,7.0,18.0,0
3,Cannock Chase Public,G,8291,78202.0,88999.0,85,41,Cannock Chase Public,"Het aanbieden van diensten aan overheden, geri...",1987.0,3.0,26.0,0
4,Sociëteit Tyche,H,88993,NaN,NaN,0,71,Sociëteit Tyche,studentenvereniging,2011.0,5.0,6.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Stichting Streekcentrum Scheerwolde,E,88999,NaN,NaN,0,74,Stichting Streekcentrum Scheerwolde,"Het oprichten, instandhouden en exploiteren va...",1966.0,3.0,4.0,1
346,Cultuur op de Campus,G,94994,NaN,NaN,0,71,Cultuur op de Campus,Het bevorderen ban een levendig cultureel klim...,2011.0,3.0,16.0,0
347,Enjoy Dagbesteding & Zorg,E,88992,NaN,NaN,2,11,Enjoy Dagbesteding & Zorg,Aanbieden van dagbesteding en zorgbesteding.,2019.0,9.0,16.0,0
348,Ontmoetingscentrum de Viskenij,A,88999,NaN,NaN,0,74,Ontmoetingscentrum de Viskenij,Ontmoetingscentrum (voor verenigingen en jeugd...,2011.0,8.0,4.0,1


In [224]:
df_KVK2022

,HN45,PROV,SBI_CODE,RECHTSVORM,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label
0,Scouting Ernst van Kempengroep,M,88993,71,0,Scouting Ernst van Kempengroep,Scoutingvereniging,2011.0,1.0,24.0,0
1,Coöperatie Duurzaam Gortel Niersen UA,G,94996,61,0,Coöperatie Duurzaam Gortel Niers,Het bijdragen aan de verduurzaming van de same...,2019.0,12.0,24.0,1
2,CVVE Winkelcentr. de Gagel UA,P,94997,61,0,Coöperatieve Vereniging van Eige,Het behartigen van de gemeenschappelijke belan...,2013.0,11.0,29.0,0
3,Coöperatie Nieuwe Energie UA,G,94997,61,0,Coöperatie Nieuwe Energie UA,Het voorzien in de stoffelijke behoeften van h...,2020.0,1.0,28.0,0
4,De Verbindingskamer,H,88999,1,1,De Verbindingskamer,Adviesburo op het gebied van welzijn en sociaa...,2022.0,1.0,12.0,0
...,...,...,...,...,...,...,...,...,...,...,...
345,Maris coaching,L,88999,1,1,Maris coaching,Coaching. Relatie coaching. Echtscheidingsmedi...,2016.0,8.0,1.0,0
346,B.V. Baekerhoes,K,88999,41,2,B.V. Baekerhoes,Het beheer en de exploitatie van gemeenschaps ...,2005.0,4.0,28.0,0
347,Lukt Beheer B.V.,M,88999,41,0,Lukt Beheer B.V.,Het beheren van stamrechtgelden alsmede financ...,2011.0,3.0,1.0,0
348,Stichting Valere Bewind,L,88999,74,0,Stichting Valere Bewind,Bewindvoering en inkomensbeheer,2013.0,11.0,14.0,0


#### Convert to appropriate datatypes

#### Check overlap between 2022, 2023 and labelled data

In [225]:

#merge the dataframes
merged_df = df_KVK2022.merge(df_labelled_drop)

#calculate the count of identical rows
identical_rows_count = len(merged_df)

#display the count of identical rows
print("Count of identical rows:", identical_rows_count)

Count of identical rows: 0


In [226]:

#merge the dataframes
merged_df = df_KVK2023.merge(df_labelled_drop)

#calculate the count of identical rows
identical_rows_count = len(merged_df)

#display the count of identical rows
print("Count of identical rows:", identical_rows_count)

Count of identical rows: 0


In [227]:
identical_rows_count = len(df_KVK2022.merge(df_KVK2023))

#display the count of identical rows
print("Count of identical rows:", identical_rows_count)

Count of identical rows: 4


#### Delete the rows that occur in the training data from the 2022+2023 data

In [228]:
#merge the dataframes to identify identical rows for 2022 data
merged_df2022 = df_KVK2022.merge(df_labelled_drop, how='left', indicator=True)

#filter out the rows that are not present in both dataframes
df_KVK2022 = merged_df2022[merged_df2022['_merge'] == 'left_only']

#drop the indicator and label column
df_KVK2022.drop('_merge', axis=1, inplace=True)

In [229]:
df_KVK2022

,HN45,PROV,SBI_CODE,RECHTSVORM,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label
0,Scouting Ernst van Kempengroep,M,88993,71,0,Scouting Ernst van Kempengroep,Scoutingvereniging,2011.0,1.0,24.0,0
1,Coöperatie Duurzaam Gortel Niersen UA,G,94996,61,0,Coöperatie Duurzaam Gortel Niers,Het bijdragen aan de verduurzaming van de same...,2019.0,12.0,24.0,1
2,CVVE Winkelcentr. de Gagel UA,P,94997,61,0,Coöperatieve Vereniging van Eige,Het behartigen van de gemeenschappelijke belan...,2013.0,11.0,29.0,0
3,Coöperatie Nieuwe Energie UA,G,94997,61,0,Coöperatie Nieuwe Energie UA,Het voorzien in de stoffelijke behoeften van h...,2020.0,1.0,28.0,0
4,De Verbindingskamer,H,88999,1,1,De Verbindingskamer,Adviesburo op het gebied van welzijn en sociaa...,2022.0,1.0,12.0,0
...,...,...,...,...,...,...,...,...,...,...,...
345,Maris coaching,L,88999,1,1,Maris coaching,Coaching. Relatie coaching. Echtscheidingsmedi...,2016.0,8.0,1.0,0
346,B.V. Baekerhoes,K,88999,41,2,B.V. Baekerhoes,Het beheer en de exploitatie van gemeenschaps ...,2005.0,4.0,28.0,0
347,Lukt Beheer B.V.,M,88999,41,0,Lukt Beheer B.V.,Het beheren van stamrechtgelden alsmede financ...,2011.0,3.0,1.0,0
348,Stichting Valere Bewind,L,88999,74,0,Stichting Valere Bewind,Bewindvoering en inkomensbeheer,2013.0,11.0,14.0,0


In [230]:
#check identical rows again
identical_rows_count = len(df_KVK2022.merge(df_labelled))

#display the count of identical rows
print("Count of identical rows:", identical_rows_count)

Count of identical rows: 0


In [231]:
#merge the dataframes to identify identical rows for 2023 data
merged_df2 = df_KVK2023.merge(df_labelled, how='left', indicator=True)

#filter out the rows that are not present in both dataframes
df_KVK2023 = merged_df2[merged_df2['_merge'] == 'left_only']

#drop the indicator and label column
df_KVK2023.drop(['_merge'], axis=1, inplace=True)

In [232]:
identical_rows_count = len(df_KVK2023.merge(df_labelled))

#display the count of identical rows
print("Count of identical rows:", identical_rows_count)

Count of identical rows: 0


# Data exploration

## Variables in the dataset

In [233]:
df_labelled['label'].value_counts()

label
0    303
1    143
Name: count, dtype: int64

The classes are slighly imbalanced. 

# Pre-processing 

#### One-hot encode SBI codes

In [234]:
#print all unique SBI codes in KVK data from 2023
df_KVK2023['SBI_CODE'].unique()

array([88992, 88999, 88991,  8291, 88993, 94993, 85599, 88102,  8720,
       94996, 88103, 85592, 88101,  9609, 86919, 94997,   161,  7810,
       94995,  8891,  9412,  6420, 72192, 74201, 94994, 70222, 70221,
       94991, 85519,  5621,   111, 74103, 94992, 86913,  1061, 47915,
       93299, 55102,  5629, 86922,  8219, 82999, 46761, 96031,  4120,
        9003, 69203,  9002, 96021, 85522, 86923, 47431,  5920,  4617,
        8010,  6202, 43221,  4639,  7490, 47291,  4932, 86104,  6190],
      dtype=int64)

In [235]:
#looping through the SBI_CODE, NEVENACT_1 and NEVENACT_2 and only keeping the SBI code from the selection based on KVK2022

#selection of important SBI codes
important_codes = {88102, 88103, 88992, 88993, 88999, 94991, 
                   94992, 94993, 94994, 94995, 94996, 94997}

#function to determine the appropriate SBI code to keep
def choose_sbi(row):
    sbi_code = None
    if row['SBI_CODE'] in important_codes:
        sbi_code = row['SBI_CODE']
    elif row['NEVENACT_1'] in important_codes:
        sbi_code = row['NEVENACT_1']
    elif row['NEVENACT_2'] in important_codes:
        sbi_code = row['NEVENACT_2']
    #convert selected SBI code to integer if not None
    return int(sbi_code) if sbi_code is not None else None

#function to apply the above logic to each row in the df
def select_important_sbi(df):
    #create a new column 'Selected_SBI' with the selected SBI code
    df['Selected_SBI'] = df.apply(choose_sbi, axis=1)
    #convert 'Selected_SBI' column to integer type
    df['Selected_SBI'] = df['Selected_SBI'].astype('Int64')  # Use 'Int64' for nullable integer type
    return df

result_df = select_important_sbi(df_KVK2023)

In [236]:
#drop the unnecessary columns SBI_CODE, NEVENACT_1, NEVENACT_2
df_KVK2023 = result_df.drop(['SBI_CODE', 'NEVENACT_1', 'NEVENACT_2'], axis=1)

In [238]:
df_KVK2023

,HN45,PROV,W_P_TOTAAL,RECHTSVORM,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label,Selected_SBI
0,De Regenboog -gespecial. residentiële zorg-,P,1,1,De Regenboog -gespecialiseerde residentiële zorg-,Het geven van ondersteunende begeleiding in de...,2008.0,10.0,16.0,0,88992
1,Dorpscentrum 'd' Ouw School',P,0,74,Dorpscentrum 'd' Ouw School',Lokaal/ruimte voor alle plaatselijke verenigin...,2010.0,8.0,3.0,0,88999
2,R.K. zorg en begeleiding,H,1,1,R.K. zorg en begeleiding,Jeugdzorg alsmede het ondersteunen en begeleid...,2018.0,7.0,18.0,0,88992
3,Cannock Chase Public,G,85,41,Cannock Chase Public,"Het aanbieden van diensten aan overheden, geri...",1987.0,3.0,26.0,0,88999
4,Sociëteit Tyche,H,0,71,Sociëteit Tyche,studentenvereniging,2011.0,5.0,6.0,0,88993
...,...,...,...,...,...,...,...,...,...,...,...
345,Stichting Streekcentrum Scheerwolde,E,0,74,Stichting Streekcentrum Scheerwolde,"Het oprichten, instandhouden en exploiteren va...",1966.0,3.0,4.0,1,88999
346,Cultuur op de Campus,G,0,71,Cultuur op de Campus,Het bevorderen ban een levendig cultureel klim...,2011.0,3.0,16.0,0,94994
347,Enjoy Dagbesteding & Zorg,E,2,11,Enjoy Dagbesteding & Zorg,Aanbieden van dagbesteding en zorgbesteding.,2019.0,9.0,16.0,0,88992
348,Ontmoetingscentrum de Viskenij,A,0,74,Ontmoetingscentrum de Viskenij,Ontmoetingscentrum (voor verenigingen en jeugd...,2011.0,8.0,4.0,1,88999


In [239]:
KVK2022 = df_KVK2022
KVK2023 = df_KVK2023
label = df_labelled

#add df identifiers to each df
KVK2022['DataFrame'] = 'KVK2022'
KVK2023['DataFrame'] = 'KVK2023'
label['DataFrame'] = 'label'

#merge the dataframes
combined_df = pd.concat([KVK2022, KVK2023, label])

#calculate overall frequency of each SBI_CODE / Selected_SBI
total_counts_sbi_code = combined_df[combined_df['DataFrame'] != 'KVK2023']['SBI_CODE'].value_counts()
total_counts_selected_sbi = combined_df[combined_df['DataFrame'] == 'KVK2023']['Selected_SBI'].value_counts()

#combine the total_counts into a single Series
total_counts = pd.concat([total_counts_sbi_code, total_counts_selected_sbi], axis=0).groupby(level=0).sum()

#calculate frequency of each SBI_CODE / Selected_SBI in each df
df_counts = pd.DataFrame(index=total_counts.index)
for df_name in ['KVK2022', 'KVK2023', 'label']:
    if df_name == 'KVK2023':
        col_name = 'Selected_SBI'
    else:
        col_name = 'SBI_CODE'
        
    df_specific_counts = combined_df[combined_df['DataFrame'] == df_name][col_name].value_counts()
    df_counts[df_name] = df_specific_counts.reindex(df_counts.index, fill_value=0)

#merge total counts and df-specific counts
merged_counts = pd.concat([total_counts.rename('Total'), df_counts], axis=1)

#sort the merged_counts DataFrame by the 'Total' column in descending order
merged_counts_sorted = merged_counts.sort_values(by='Total', ascending=False)

#display the whole merged counts table sorted
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(merged_counts_sorted)

,Total,KVK2022,KVK2023,label
88999.0,261,80,87,94
88993.0,216,77,51,88
94997.0,197,63,40,94
88992.0,178,48,79,51
88103.0,81,29,34,18
88102.0,43,9,23,11
94993.0,30,9,6,15
94996.0,28,11,7,10
94995.0,24,8,5,11
94992.0,23,7,8,8


The table shows that below a frequency of 162, the sbi codes only occur in the training dataset and not the KVK2022 or the KVK2023. So, we decided to group all the sbi codes with a frequency below 100 to minimize the complexity of the model after one-hot encoding this variable.

In [240]:
df_KVK2022['SBI_CODE'].unique()

array([88993, 94996, 94997, 88999, 94994, 88103, 88992, 94991, 94993,
       88102, 94995, 94992], dtype=int64)

In [241]:
df_KVK2023['Selected_SBI'].unique()

<IntegerArray>
[88992, 88999, 88993, 94993, 88102, 94996, 88103, 94997, 94995, 94994, 94991,
 94992]
Length: 12, dtype: Int64

In [242]:
#one-hot encode the 'Selected_SBI' column in KVK2023
one_hot_encoded = pd.get_dummies(df_KVK2023['Selected_SBI'], prefix='SBI')

#convert boolean values to integers (0 or 1)
one_hot_encoded = one_hot_encoded.astype(int)

#concatenate the one-hot encoded df with the original df
df_KVK2023_encoded = pd.concat([df_KVK2023, one_hot_encoded], axis=1)

#drop the unnecessary columns Selected_SBI and DataFrame
df_KVK2023_encoded = df_KVK2023_encoded.drop(['Selected_SBI', 'DataFrame'], axis=1)

#add empty column at the end to match the column in the training dataset df_labelled
df_KVK2023_encoded['SBI_overig'] = 0

df_KVK2023_encoded

,HN45,PROV,W_P_TOTAAL,RECHTSVORM,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label,...,SBI_88993,SBI_88999,SBI_94991,SBI_94992,SBI_94993,SBI_94994,SBI_94995,SBI_94996,SBI_94997,SBI_overig
0,De Regenboog -gespecial. residentiële zorg-,P,1,1,De Regenboog -gespecialiseerde residentiële zorg-,Het geven van ondersteunende begeleiding in de...,2008.0,10.0,16.0,0,...,0,0,0,0,0,0,0,0,0,0
1,Dorpscentrum 'd' Ouw School',P,0,74,Dorpscentrum 'd' Ouw School',Lokaal/ruimte voor alle plaatselijke verenigin...,2010.0,8.0,3.0,0,...,0,1,0,0,0,0,0,0,0,0
2,R.K. zorg en begeleiding,H,1,1,R.K. zorg en begeleiding,Jeugdzorg alsmede het ondersteunen en begeleid...,2018.0,7.0,18.0,0,...,0,0,0,0,0,0,0,0,0,0
3,Cannock Chase Public,G,85,41,Cannock Chase Public,"Het aanbieden van diensten aan overheden, geri...",1987.0,3.0,26.0,0,...,0,1,0,0,0,0,0,0,0,0
4,Sociëteit Tyche,H,0,71,Sociëteit Tyche,studentenvereniging,2011.0,5.0,6.0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Stichting Streekcentrum Scheerwolde,E,0,74,Stichting Streekcentrum Scheerwolde,"Het oprichten, instandhouden en exploiteren va...",1966.0,3.0,4.0,1,...,0,1,0,0,0,0,0,0,0,0
346,Cultuur op de Campus,G,0,71,Cultuur op de Campus,Het bevorderen ban een levendig cultureel klim...,2011.0,3.0,16.0,0,...,0,0,0,0,0,1,0,0,0,0
347,Enjoy Dagbesteding & Zorg,E,2,11,Enjoy Dagbesteding & Zorg,Aanbieden van dagbesteding en zorgbesteding.,2019.0,9.0,16.0,0,...,0,0,0,0,0,0,0,0,0,0
348,Ontmoetingscentrum de Viskenij,A,0,74,Ontmoetingscentrum de Viskenij,Ontmoetingscentrum (voor verenigingen en jeugd...,2011.0,8.0,4.0,1,...,0,1,0,0,0,0,0,0,0,0


In [243]:
#one-hot encode the 'SBI_CODE' column in KVK2022
one_hot_encoded = pd.get_dummies(df_KVK2022['SBI_CODE'], prefix='SBI')

#convert boolean values to integers (0 or 1)
one_hot_encoded = one_hot_encoded.astype(int)

#concatenate the one-hot encoded df with the original df
df_KVK2022_encoded = pd.concat([df_KVK2022, one_hot_encoded], axis=1)

#drop the unnecessary columns SBI_CODE and DataFrame
df_KVK2022_encoded = df_KVK2022_encoded.drop(['SBI_CODE', 'DataFrame'], axis=1)

#add empty column at the end to match the column in the training dataset df_labelled
df_KVK2022_encoded['SBI_overig'] = 0

df_KVK2022_encoded

,HN45,PROV,RECHTSVORM,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label,...,SBI_88993,SBI_88999,SBI_94991,SBI_94992,SBI_94993,SBI_94994,SBI_94995,SBI_94996,SBI_94997,SBI_overig
0,Scouting Ernst van Kempengroep,M,71,0,Scouting Ernst van Kempengroep,Scoutingvereniging,2011.0,1.0,24.0,0,...,1,0,0,0,0,0,0,0,0,0
1,Coöperatie Duurzaam Gortel Niersen UA,G,61,0,Coöperatie Duurzaam Gortel Niers,Het bijdragen aan de verduurzaming van de same...,2019.0,12.0,24.0,1,...,0,0,0,0,0,0,0,1,0,0
2,CVVE Winkelcentr. de Gagel UA,P,61,0,Coöperatieve Vereniging van Eige,Het behartigen van de gemeenschappelijke belan...,2013.0,11.0,29.0,0,...,0,0,0,0,0,0,0,0,1,0
3,Coöperatie Nieuwe Energie UA,G,61,0,Coöperatie Nieuwe Energie UA,Het voorzien in de stoffelijke behoeften van h...,2020.0,1.0,28.0,0,...,0,0,0,0,0,0,0,0,1,0
4,De Verbindingskamer,H,1,1,De Verbindingskamer,Adviesburo op het gebied van welzijn en sociaa...,2022.0,1.0,12.0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Maris coaching,L,1,1,Maris coaching,Coaching. Relatie coaching. Echtscheidingsmedi...,2016.0,8.0,1.0,0,...,0,1,0,0,0,0,0,0,0,0
346,B.V. Baekerhoes,K,41,2,B.V. Baekerhoes,Het beheer en de exploitatie van gemeenschaps ...,2005.0,4.0,28.0,0,...,0,1,0,0,0,0,0,0,0,0
347,Lukt Beheer B.V.,M,41,0,Lukt Beheer B.V.,Het beheren van stamrechtgelden alsmede financ...,2011.0,3.0,1.0,0,...,0,1,0,0,0,0,0,0,0,0
348,Stichting Valere Bewind,L,74,0,Stichting Valere Bewind,Bewindvoering en inkomensbeheer,2013.0,11.0,14.0,0,...,0,1,0,0,0,0,0,0,0,0


In [244]:
#one-hot encode the SBI column in df_labelled while grouping the codes with a frequency below 100

#merge the DataFrames with identifiers
df_KVK2022['DataFrame'] = 'KVK2022'
df_KVK2023['DataFrame'] = 'KVK2023'
df_labelled['DataFrame'] = 'labelled'

#concatenate the dataframes
combined_df = pd.concat([df_KVK2022, df_KVK2023, df_labelled])

#calculate overall frequency of each SBI_CODE / Selected_SBI
total_counts_sbi_code = combined_df[combined_df['DataFrame'] != 'KVK2023']['SBI_CODE'].value_counts()
total_counts_selected_sbi = combined_df[combined_df['DataFrame'] == 'KVK2023']['Selected_SBI'].value_counts()

#combine the total_counts into a single Series
total_counts = pd.concat([total_counts_sbi_code, total_counts_selected_sbi], axis=0).groupby(level=0).sum()

#identify SBI codes with frequency below 100 and group them as 'overig'
low_freq_sbis = total_counts[total_counts < 100].index
combined_df.loc[combined_df['SBI_CODE'].isin(low_freq_sbis), 'SBI_CODE'] = 'overig'

#initialize df to store frequency counts for each category
df_counts = pd.DataFrame(index=combined_df['SBI_CODE'].unique())

#calculate frequency of each SBI_CODE / Selected_SBI in each df after grouping low-frequency SBIs
for df_name in ['KVK2022', 'KVK2023', 'labelled']:
    if df_name == 'KVK2023':
        col_name = 'Selected_SBI'
    else:
        col_name = 'SBI_CODE'
    
    #filter rows for the specific DataFrame
    df_specific = combined_df[combined_df['DataFrame'] == df_name]
    
    #calculate value counts for the specific column
    df_specific_counts = df_specific[col_name].value_counts()
    
    #update df_counts with specific counts (reindex to include all unique categories)
    df_counts[df_name] = df_specific_counts.reindex(df_counts.index, fill_value=0)

#add 'overig' category with zeros to df_counts if it doesn't exist
if 'overig' not in df_counts.index:
    df_counts.loc['overig'] = 0

#sort the merged counts df by the 'Total' column in descending order
merged_counts_sorted = total_counts.rename('Total').sort_values(ascending=False)

#display the whole merged counts table sorted
#with pd.option_context('display.max_rows', None):
#    display(merged_counts_sorted)

#one-hot encode the 'SBI_CODE' column in df_labelled (after grouping 'overig')
one_hot_encoded = pd.get_dummies(df_labelled['SBI_CODE'].replace(low_freq_sbis, 'overig'), prefix='SBI')

#convert boolean values to integers (0 or 1)
one_hot_encoded = one_hot_encoded.astype(int)

#concatenate the one-hot encoded DataFrame with the original DataFrame (df_labelled)
df_labelled_encoded = pd.concat([df_labelled, one_hot_encoded], axis=1)

#drop the unnecessary columns SBI_CODE and DataFrame
df_labelled_encoded = df_labelled_encoded.drop(['SBI_CODE', 'DataFrame'], axis=1)

df_labelled_encoded

C:\Users\SaidKalee\AppData\Local\Temp\ipykernel_26868\3573732819.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'overig' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combined_df.loc[combined_df['SBI_CODE'].isin(low_freq_sbis), 'SBI_CODE'] = 'overig'


,HN45,PROV,RECHTSVORM,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,label,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,SBI_88992,SBI_88993,SBI_88999,SBI_94997,SBI_overig
3,Maatschap Beheer MFC Tiendeveen,D,7,0,Maatschap Beheer MFC Tiendeveen,Het beheren van het Multifunctioneel Centrum i...,1,2013,3,14,0,0,1,0,0
4,Aktiviteitencentrum Alletahof,H,71,0,Aktiviteitencentrum Alletahof,Buurt- en clubhuiswerk en samenlevingsopbouwwerk,1,2011,2,16,0,1,0,0,0
5,Coöperatie Stationspark Deurne UA,P,61,0,Coöperatie Stationspark Deurne U,"De kwaliteit van de woon-, werk- en leefomgevi...",1,2014,12,23,0,0,0,1,0
6,Coöperatie Sustar UA,G,61,0,Coöperatie Sustar UA,Het bieden van ondersteuning aan organisaties ...,1,2022,7,1,0,0,0,1,0
7,"Jasmijnsteede, Centraal wonen voor ouderen",D,71,0,"Jasmijnsteede, Centraal wonen vo",Groepswonen voor ouderen.,1,2011,4,20,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,Verenig. Groepsw. van Oud. 'Ouder-Landshoek',H,71,0,Vereniging Groepswonen van Ouder,Bejaardenverenigingen,1,1990,3,6,0,0,0,0,1
253,Bewonersvereniging Centraal Wonen 'De Banier',H,71,0,Bewonersvereniging Centraal Wone,Verhuur van onroerend goed,1,1979,10,26,0,0,0,0,1
257,Stichting Dorpsraad Ven-Zelderheide,K,74,5,Stichting Dorpsraad Ven-Zelderhe,"Belangenverenigingen, meer bezighouden met ge-...",1,1981,6,26,0,0,0,1,0
262,Stichting Wijkraad Loven-Besterd,P,74,0,Stichting Wijkraad Loven-Besterd,Buurthuiswerk en samenlevingsopbouwwerk,1,1988,7,4,0,1,0,0,0


#### One-hot encode PROV (provincie)

In [245]:
#one-hot encode the 'PROV' column in KVK2022
one_hot_encoded = pd.get_dummies(df_KVK2022_encoded['PROV'], prefix='PROV')

#convert boolean values to integers (0 or 1)
one_hot_encoded = one_hot_encoded.astype(int)

#concatenate the one-hot encoded df with the original df
df_KVK2022_encoded = pd.concat([df_KVK2022_encoded, one_hot_encoded], axis=1)

#drop the unnecessary column PROV
df_KVK2022_encoded = df_KVK2022_encoded.drop(['PROV'], axis=1)

df_KVK2022_encoded

,HN45,RECHTSVORM,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label,SBI_88102,...,PROV_D,PROV_E,PROV_G,PROV_H,PROV_K,PROV_L,PROV_M,PROV_P,PROV_S,PROV_X
0,Scouting Ernst van Kempengroep,71,0,Scouting Ernst van Kempengroep,Scoutingvereniging,2011.0,1.0,24.0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,Coöperatie Duurzaam Gortel Niersen UA,61,0,Coöperatie Duurzaam Gortel Niers,Het bijdragen aan de verduurzaming van de same...,2019.0,12.0,24.0,1,0,...,0,0,1,0,0,0,0,0,0,0
2,CVVE Winkelcentr. de Gagel UA,61,0,Coöperatieve Vereniging van Eige,Het behartigen van de gemeenschappelijke belan...,2013.0,11.0,29.0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Coöperatie Nieuwe Energie UA,61,0,Coöperatie Nieuwe Energie UA,Het voorzien in de stoffelijke behoeften van h...,2020.0,1.0,28.0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,De Verbindingskamer,1,1,De Verbindingskamer,Adviesburo op het gebied van welzijn en sociaa...,2022.0,1.0,12.0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Maris coaching,1,1,Maris coaching,Coaching. Relatie coaching. Echtscheidingsmedi...,2016.0,8.0,1.0,0,0,...,0,0,0,0,0,1,0,0,0,0
346,B.V. Baekerhoes,41,2,B.V. Baekerhoes,Het beheer en de exploitatie van gemeenschaps ...,2005.0,4.0,28.0,0,0,...,0,0,0,0,1,0,0,0,0,0
347,Lukt Beheer B.V.,41,0,Lukt Beheer B.V.,Het beheren van stamrechtgelden alsmede financ...,2011.0,3.0,1.0,0,0,...,0,0,0,0,0,0,1,0,0,0
348,Stichting Valere Bewind,74,0,Stichting Valere Bewind,Bewindvoering en inkomensbeheer,2013.0,11.0,14.0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [246]:
#one-hot encode the 'PROV' column in KVK2023
one_hot_encoded = pd.get_dummies(df_KVK2023_encoded['PROV'], prefix='PROV')

#convert boolean values to integers (0 or 1)
one_hot_encoded = one_hot_encoded.astype(int)

#concatenate the one-hot encoded df with the original df
df_KVK2023_encoded = pd.concat([df_KVK2023_encoded, one_hot_encoded], axis=1)

#drop the unnecessary column PROV
df_KVK2023_encoded = df_KVK2023_encoded.drop(['PROV'], axis=1)

df_KVK2023_encoded

,HN45,W_P_TOTAAL,RECHTSVORM,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label,SBI_88102,...,PROV_D,PROV_E,PROV_G,PROV_H,PROV_K,PROV_L,PROV_M,PROV_P,PROV_S,PROV_X
0,De Regenboog -gespecial. residentiële zorg-,1,1,De Regenboog -gespecialiseerde residentiële zorg-,Het geven van ondersteunende begeleiding in de...,2008.0,10.0,16.0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,Dorpscentrum 'd' Ouw School',0,74,Dorpscentrum 'd' Ouw School',Lokaal/ruimte voor alle plaatselijke verenigin...,2010.0,8.0,3.0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,R.K. zorg en begeleiding,1,1,R.K. zorg en begeleiding,Jeugdzorg alsmede het ondersteunen en begeleid...,2018.0,7.0,18.0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Cannock Chase Public,85,41,Cannock Chase Public,"Het aanbieden van diensten aan overheden, geri...",1987.0,3.0,26.0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Sociëteit Tyche,0,71,Sociëteit Tyche,studentenvereniging,2011.0,5.0,6.0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Stichting Streekcentrum Scheerwolde,0,74,Stichting Streekcentrum Scheerwolde,"Het oprichten, instandhouden en exploiteren va...",1966.0,3.0,4.0,1,0,...,0,1,0,0,0,0,0,0,0,0
346,Cultuur op de Campus,0,71,Cultuur op de Campus,Het bevorderen ban een levendig cultureel klim...,2011.0,3.0,16.0,0,0,...,0,0,1,0,0,0,0,0,0,0
347,Enjoy Dagbesteding & Zorg,2,11,Enjoy Dagbesteding & Zorg,Aanbieden van dagbesteding en zorgbesteding.,2019.0,9.0,16.0,0,0,...,0,1,0,0,0,0,0,0,0,0
348,Ontmoetingscentrum de Viskenij,0,74,Ontmoetingscentrum de Viskenij,Ontmoetingscentrum (voor verenigingen en jeugd...,2011.0,8.0,4.0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [247]:
#one-hot encode the 'PROV' column in df_labelled
one_hot_encoded = pd.get_dummies(df_labelled_encoded['PROV'], prefix='PROV')

#convert boolean values to integers (0 or 1)
one_hot_encoded = one_hot_encoded.astype(int)

#concatenate the one-hot encoded df with the original df
df_labelled_encoded = pd.concat([df_labelled_encoded, one_hot_encoded], axis=1)

#drop the unnecessary column PROV
df_labelled_encoded = df_labelled_encoded.drop(['PROV'], axis=1)

df_labelled_encoded

,HN45,RECHTSVORM,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,label,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,SBI_88992,...,PROV_D,PROV_E,PROV_G,PROV_H,PROV_K,PROV_L,PROV_M,PROV_P,PROV_S,PROV_X
3,Maatschap Beheer MFC Tiendeveen,7,0,Maatschap Beheer MFC Tiendeveen,Het beheren van het Multifunctioneel Centrum i...,1,2013,3,14,0,...,1,0,0,0,0,0,0,0,0,0
4,Aktiviteitencentrum Alletahof,71,0,Aktiviteitencentrum Alletahof,Buurt- en clubhuiswerk en samenlevingsopbouwwerk,1,2011,2,16,0,...,0,0,0,1,0,0,0,0,0,0
5,Coöperatie Stationspark Deurne UA,61,0,Coöperatie Stationspark Deurne U,"De kwaliteit van de woon-, werk- en leefomgevi...",1,2014,12,23,0,...,0,0,0,0,0,0,0,1,0,0
6,Coöperatie Sustar UA,61,0,Coöperatie Sustar UA,Het bieden van ondersteuning aan organisaties ...,1,2022,7,1,0,...,0,0,1,0,0,0,0,0,0,0
7,"Jasmijnsteede, Centraal wonen voor ouderen",71,0,"Jasmijnsteede, Centraal wonen vo",Groepswonen voor ouderen.,1,2011,4,20,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,Verenig. Groepsw. van Oud. 'Ouder-Landshoek',71,0,Vereniging Groepswonen van Ouder,Bejaardenverenigingen,1,1990,3,6,0,...,0,0,0,1,0,0,0,0,0,0
253,Bewonersvereniging Centraal Wonen 'De Banier',71,0,Bewonersvereniging Centraal Wone,Verhuur van onroerend goed,1,1979,10,26,0,...,0,0,0,1,0,0,0,0,0,0
257,Stichting Dorpsraad Ven-Zelderheide,74,5,Stichting Dorpsraad Ven-Zelderhe,"Belangenverenigingen, meer bezighouden met ge-...",1,1981,6,26,0,...,0,0,0,0,1,0,0,0,0,0
262,Stichting Wijkraad Loven-Besterd,74,0,Stichting Wijkraad Loven-Besterd,Buurthuiswerk en samenlevingsopbouwwerk,1,1988,7,4,0,...,0,0,0,0,0,0,0,1,0,0


In [248]:
df_labelled_encoded

,HN45,RECHTSVORM,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,label,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,SBI_88992,...,PROV_D,PROV_E,PROV_G,PROV_H,PROV_K,PROV_L,PROV_M,PROV_P,PROV_S,PROV_X
3,Maatschap Beheer MFC Tiendeveen,7,0,Maatschap Beheer MFC Tiendeveen,Het beheren van het Multifunctioneel Centrum i...,1,2013,3,14,0,...,1,0,0,0,0,0,0,0,0,0
4,Aktiviteitencentrum Alletahof,71,0,Aktiviteitencentrum Alletahof,Buurt- en clubhuiswerk en samenlevingsopbouwwerk,1,2011,2,16,0,...,0,0,0,1,0,0,0,0,0,0
5,Coöperatie Stationspark Deurne UA,61,0,Coöperatie Stationspark Deurne U,"De kwaliteit van de woon-, werk- en leefomgevi...",1,2014,12,23,0,...,0,0,0,0,0,0,0,1,0,0
6,Coöperatie Sustar UA,61,0,Coöperatie Sustar UA,Het bieden van ondersteuning aan organisaties ...,1,2022,7,1,0,...,0,0,1,0,0,0,0,0,0,0
7,"Jasmijnsteede, Centraal wonen voor ouderen",71,0,"Jasmijnsteede, Centraal wonen vo",Groepswonen voor ouderen.,1,2011,4,20,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,Verenig. Groepsw. van Oud. 'Ouder-Landshoek',71,0,Vereniging Groepswonen van Ouder,Bejaardenverenigingen,1,1990,3,6,0,...,0,0,0,1,0,0,0,0,0,0
253,Bewonersvereniging Centraal Wonen 'De Banier',71,0,Bewonersvereniging Centraal Wone,Verhuur van onroerend goed,1,1979,10,26,0,...,0,0,0,1,0,0,0,0,0,0
257,Stichting Dorpsraad Ven-Zelderheide,74,5,Stichting Dorpsraad Ven-Zelderhe,"Belangenverenigingen, meer bezighouden met ge-...",1,1981,6,26,0,...,0,0,0,0,1,0,0,0,0,0
262,Stichting Wijkraad Loven-Besterd,74,0,Stichting Wijkraad Loven-Besterd,Buurthuiswerk en samenlevingsopbouwwerk,1,1988,7,4,0,...,0,0,0,0,0,0,0,1,0,0


#### One-hot encode rechtsvorm

In [249]:
df_labelled_encoded["RECHTSVORM"].nunique()
df_KVK2022_encoded["RECHTSVORM"].nunique()
df_KVK2023_encoded["RECHTSVORM"].nunique()

9

In [250]:
KVK2022 = df_KVK2022_encoded
KVK2023 = df_KVK2023_encoded
label = df_labelled_encoded

#add df identifiers to each df
KVK2022['DataFrame'] = 'KVK2022'
KVK2023['DataFrame'] = 'KVK2023'
label['DataFrame'] = 'label'

#merge the dataframes
combined_df = pd.concat([KVK2022, KVK2023, label])

#calculate overall frequency of each PROV
total_counts_prov = combined_df['RECHTSVORM'].value_counts()

#calculate frequency of each PROV in each df
df_counts = pd.DataFrame(index=total_counts_prov.index)
for df_name in ['KVK2022', 'KVK2023', 'label']:
    df_specific_counts = combined_df[combined_df['DataFrame'] == df_name]['RECHTSVORM'].value_counts()
    df_counts[df_name] = df_specific_counts.reindex(df_counts.index, fill_value=0)

#merge total counts and df-specific counts
merged_counts = pd.concat([total_counts_prov.rename('Total'), df_counts], axis=1)

#sort the merged_counts df by the 'Total' column in descending order
merged_counts_sorted = merged_counts.sort_values(by='Total', ascending=False)

#display the whole merged counts table sorted
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(merged_counts_sorted)

,Total,KVK2022,KVK2023,label
RECHTSVORM,,,,
1,372,120,171,81
74,366,99,77,190
61,172,49,30,93
71,96,40,25,31
41,93,27,32,34
11,29,8,11,10
7,6,0,0,6
72,4,2,2,0
94,2,1,1,0


In [251]:
#combine all dataframes into one
all_dataframes = [df_KVK2022_encoded, df_KVK2023_encoded, df_labelled_encoded]
combined_df = pd.concat(all_dataframes)

#define the frequency threshold
frequency_threshold = 10

#calculate the overall frequency of each RECHTSVORM
total_counts = combined_df['RECHTSVORM'].value_counts()

#get the categories with frequency over the threshold
freq_over_threshold = total_counts[total_counts >= frequency_threshold].index.tolist()

#create a new column 'RECHTSVORM_overig' to store categories with frequency below the threshold
combined_df['overig'] = combined_df['RECHTSVORM'].apply(lambda x: x if x in freq_over_threshold else 'overig')

#perform one-hot encoding
one_hot_encoded = pd.get_dummies(combined_df['overig'])

#convert boolean values to integers (0 or 1)
one_hot_encoded = one_hot_encoded.astype(int)

#rename columns with the prefix 'rechtsvorm'
one_hot_encoded.columns = ['rechtsvorm_' + str(col) for col in one_hot_encoded.columns]

#combine the original dataframe with the one-hot encoded dataframe
combined_df = pd.concat([combined_df, one_hot_encoded], axis=1)

#drop unnecessary columns
combined_df.drop(['RECHTSVORM', 'overig'], axis=1, inplace=True)

#split the combined dataframe back into the original dataframes with the same columns
df_KVK2022_encoded_result = combined_df[combined_df['DataFrame'] == 'KVK2022'].drop('DataFrame', axis=1)
df_KVK2023_encoded_result = combined_df[combined_df['DataFrame'] == 'KVK2023'].drop('DataFrame', axis=1)
df_labelled_encoded_result = combined_df[combined_df['DataFrame'] == 'label'].drop('DataFrame', axis=1)

In [252]:
df_KVK2023_encoded_result.columns

Index(['HN45', 'W_P_TOTAAL', 'H_NAAM_VOL', 'Bedrijfsomschrijving',
       'INSCHR_DAT_JAAR', 'INSCHR_DAT_MAAND', 'INSCHR_DAT_DAG', 'label',
       'SBI_88102', 'SBI_88103', 'SBI_88992', 'SBI_88993', 'SBI_88999',
       'SBI_94991', 'SBI_94992', 'SBI_94993', 'SBI_94994', 'SBI_94995',
       'SBI_94996', 'SBI_94997', 'SBI_overig', 'PROV_A', 'PROV_B', 'PROV_D',
       'PROV_E', 'PROV_G', 'PROV_H', 'PROV_K', 'PROV_L', 'PROV_M', 'PROV_P',
       'PROV_S', 'PROV_X', 'rechtsvorm_1', 'rechtsvorm_11', 'rechtsvorm_41',
       'rechtsvorm_61', 'rechtsvorm_71', 'rechtsvorm_74', 'rechtsvorm_overig'],
      dtype='object')

In [253]:
df_KVK2022_encoded_result.columns

Index(['HN45', 'W_P_TOTAAL', 'H_NAAM_VOL', 'Bedrijfsomschrijving',
       'INSCHR_DAT_JAAR', 'INSCHR_DAT_MAAND', 'INSCHR_DAT_DAG', 'label',
       'SBI_88102', 'SBI_88103', 'SBI_88992', 'SBI_88993', 'SBI_88999',
       'SBI_94991', 'SBI_94992', 'SBI_94993', 'SBI_94994', 'SBI_94995',
       'SBI_94996', 'SBI_94997', 'SBI_overig', 'PROV_A', 'PROV_B', 'PROV_D',
       'PROV_E', 'PROV_G', 'PROV_H', 'PROV_K', 'PROV_L', 'PROV_M', 'PROV_P',
       'PROV_S', 'PROV_X', 'rechtsvorm_1', 'rechtsvorm_11', 'rechtsvorm_41',
       'rechtsvorm_61', 'rechtsvorm_71', 'rechtsvorm_74', 'rechtsvorm_overig'],
      dtype='object')

In [254]:
df_labelled_encoded_result.columns

Index(['HN45', 'W_P_TOTAAL', 'H_NAAM_VOL', 'Bedrijfsomschrijving',
       'INSCHR_DAT_JAAR', 'INSCHR_DAT_MAAND', 'INSCHR_DAT_DAG', 'label',
       'SBI_88102', 'SBI_88103', 'SBI_88992', 'SBI_88993', 'SBI_88999',
       'SBI_94991', 'SBI_94992', 'SBI_94993', 'SBI_94994', 'SBI_94995',
       'SBI_94996', 'SBI_94997', 'SBI_overig', 'PROV_A', 'PROV_B', 'PROV_D',
       'PROV_E', 'PROV_G', 'PROV_H', 'PROV_K', 'PROV_L', 'PROV_M', 'PROV_P',
       'PROV_S', 'PROV_X', 'rechtsvorm_1', 'rechtsvorm_11', 'rechtsvorm_41',
       'rechtsvorm_61', 'rechtsvorm_71', 'rechtsvorm_74', 'rechtsvorm_overig'],
      dtype='object')

In [255]:
#list of new columns to add , these columns occur in the original training data but are not available in the kvk2023/kvk2022 data
new_columns = ['rechtsvorm_7', 'rechtsvorm_21', 'rechtsvorm_70', 'rechtsvorm_72', 'rechtsvorm_88']

#add new columns with 0 values
for col in new_columns:
    df_KVK2022_encoded_result[col] = 0

#display the updated DataFrame
df_KVK2022_encoded_result

,HN45,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label,SBI_88102,SBI_88103,...,rechtsvorm_41,rechtsvorm_61,rechtsvorm_71,rechtsvorm_74,rechtsvorm_overig,rechtsvorm_7,rechtsvorm_21,rechtsvorm_70,rechtsvorm_72,rechtsvorm_88
0,Scouting Ernst van Kempengroep,0,Scouting Ernst van Kempengroep,Scoutingvereniging,2011.0,1.0,24.0,0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
1,Coöperatie Duurzaam Gortel Niersen UA,0,Coöperatie Duurzaam Gortel Niers,Het bijdragen aan de verduurzaming van de same...,2019.0,12.0,24.0,1,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,CVVE Winkelcentr. de Gagel UA,0,Coöperatieve Vereniging van Eige,Het behartigen van de gemeenschappelijke belan...,2013.0,11.0,29.0,0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
3,Coöperatie Nieuwe Energie UA,0,Coöperatie Nieuwe Energie UA,Het voorzien in de stoffelijke behoeften van h...,2020.0,1.0,28.0,0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
4,De Verbindingskamer,1,De Verbindingskamer,Adviesburo op het gebied van welzijn en sociaa...,2022.0,1.0,12.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Maris coaching,1,Maris coaching,Coaching. Relatie coaching. Echtscheidingsmedi...,2016.0,8.0,1.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
346,B.V. Baekerhoes,2,B.V. Baekerhoes,Het beheer en de exploitatie van gemeenschaps ...,2005.0,4.0,28.0,0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
347,Lukt Beheer B.V.,0,Lukt Beheer B.V.,Het beheren van stamrechtgelden alsmede financ...,2011.0,3.0,1.0,0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
348,Stichting Valere Bewind,0,Stichting Valere Bewind,Bewindvoering en inkomensbeheer,2013.0,11.0,14.0,0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


In [256]:
#same for 2023
for col in new_columns:
    df_KVK2023_encoded_result[col] = 0

In [257]:
df_KVK2022 = df_KVK2022_encoded_result
df_KVK2023 = df_KVK2023_encoded_result
df_labelled = df_labelled_encoded_result

In [258]:
df_KVK2022

,HN45,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label,SBI_88102,SBI_88103,...,rechtsvorm_41,rechtsvorm_61,rechtsvorm_71,rechtsvorm_74,rechtsvorm_overig,rechtsvorm_7,rechtsvorm_21,rechtsvorm_70,rechtsvorm_72,rechtsvorm_88
0,Scouting Ernst van Kempengroep,0,Scouting Ernst van Kempengroep,Scoutingvereniging,2011.0,1.0,24.0,0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
1,Coöperatie Duurzaam Gortel Niersen UA,0,Coöperatie Duurzaam Gortel Niers,Het bijdragen aan de verduurzaming van de same...,2019.0,12.0,24.0,1,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,CVVE Winkelcentr. de Gagel UA,0,Coöperatieve Vereniging van Eige,Het behartigen van de gemeenschappelijke belan...,2013.0,11.0,29.0,0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
3,Coöperatie Nieuwe Energie UA,0,Coöperatie Nieuwe Energie UA,Het voorzien in de stoffelijke behoeften van h...,2020.0,1.0,28.0,0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
4,De Verbindingskamer,1,De Verbindingskamer,Adviesburo op het gebied van welzijn en sociaa...,2022.0,1.0,12.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Maris coaching,1,Maris coaching,Coaching. Relatie coaching. Echtscheidingsmedi...,2016.0,8.0,1.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
346,B.V. Baekerhoes,2,B.V. Baekerhoes,Het beheer en de exploitatie van gemeenschaps ...,2005.0,4.0,28.0,0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
347,Lukt Beheer B.V.,0,Lukt Beheer B.V.,Het beheren van stamrechtgelden alsmede financ...,2011.0,3.0,1.0,0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
348,Stichting Valere Bewind,0,Stichting Valere Bewind,Bewindvoering en inkomensbeheer,2013.0,11.0,14.0,0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


In [259]:
df_KVK2023

,HN45,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label,SBI_88102,SBI_88103,...,rechtsvorm_41,rechtsvorm_61,rechtsvorm_71,rechtsvorm_74,rechtsvorm_overig,rechtsvorm_7,rechtsvorm_21,rechtsvorm_70,rechtsvorm_72,rechtsvorm_88
0,De Regenboog -gespecial. residentiële zorg-,1,De Regenboog -gespecialiseerde residentiële zorg-,Het geven van ondersteunende begeleiding in de...,2008.0,10.0,16.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,Dorpscentrum 'd' Ouw School',0,Dorpscentrum 'd' Ouw School',Lokaal/ruimte voor alle plaatselijke verenigin...,2010.0,8.0,3.0,0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
2,R.K. zorg en begeleiding,1,R.K. zorg en begeleiding,Jeugdzorg alsmede het ondersteunen en begeleid...,2018.0,7.0,18.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,Cannock Chase Public,85,Cannock Chase Public,"Het aanbieden van diensten aan overheden, geri...",1987.0,3.0,26.0,0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
4,Sociëteit Tyche,0,Sociëteit Tyche,studentenvereniging,2011.0,5.0,6.0,0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,Stichting Streekcentrum Scheerwolde,0,Stichting Streekcentrum Scheerwolde,"Het oprichten, instandhouden en exploiteren va...",1966.0,3.0,4.0,1,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
346,Cultuur op de Campus,0,Cultuur op de Campus,Het bevorderen ban een levendig cultureel klim...,2011.0,3.0,16.0,0,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
347,Enjoy Dagbesteding & Zorg,2,Enjoy Dagbesteding & Zorg,Aanbieden van dagbesteding en zorgbesteding.,2019.0,9.0,16.0,0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
348,Ontmoetingscentrum de Viskenij,0,Ontmoetingscentrum de Viskenij,Ontmoetingscentrum (voor verenigingen en jeugd...,2011.0,8.0,4.0,1,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


In [260]:
df_labelled

,HN45,W_P_TOTAAL,H_NAAM_VOL,Bedrijfsomschrijving,INSCHR_DAT_JAAR,INSCHR_DAT_MAAND,INSCHR_DAT_DAG,label,SBI_88102,SBI_88103,...,PROV_P,PROV_S,PROV_X,rechtsvorm_1,rechtsvorm_11,rechtsvorm_41,rechtsvorm_61,rechtsvorm_71,rechtsvorm_74,rechtsvorm_overig
3,Maatschap Beheer MFC Tiendeveen,0,Maatschap Beheer MFC Tiendeveen,Het beheren van het Multifunctioneel Centrum i...,2013.0,3.0,14.0,1,NaN,NaN,...,0,0,0,0,0,0,0,0,0,1
4,Aktiviteitencentrum Alletahof,0,Aktiviteitencentrum Alletahof,Buurt- en clubhuiswerk en samenlevingsopbouwwerk,2011.0,2.0,16.0,1,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
5,Coöperatie Stationspark Deurne UA,0,Coöperatie Stationspark Deurne U,"De kwaliteit van de woon-, werk- en leefomgevi...",2014.0,12.0,23.0,1,NaN,NaN,...,1,0,0,0,0,0,1,0,0,0
6,Coöperatie Sustar UA,0,Coöperatie Sustar UA,Het bieden van ondersteuning aan organisaties ...,2022.0,7.0,1.0,1,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
7,"Jasmijnsteede, Centraal wonen voor ouderen",0,"Jasmijnsteede, Centraal wonen vo",Groepswonen voor ouderen.,2011.0,4.0,20.0,1,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,Verenig. Groepsw. van Oud. 'Ouder-Landshoek',0,Vereniging Groepswonen van Ouder,Bejaardenverenigingen,1990.0,3.0,6.0,1,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
253,Bewonersvereniging Centraal Wonen 'De Banier',0,Bewonersvereniging Centraal Wone,Verhuur van onroerend goed,1979.0,10.0,26.0,1,NaN,NaN,...,0,0,0,0,0,0,0,1,0,0
257,Stichting Dorpsraad Ven-Zelderheide,5,Stichting Dorpsraad Ven-Zelderhe,"Belangenverenigingen, meer bezighouden met ge-...",1981.0,6.0,26.0,1,NaN,NaN,...,0,0,0,0,0,0,0,0,1,0
262,Stichting Wijkraad Loven-Besterd,0,Stichting Wijkraad Loven-Besterd,Buurthuiswerk en samenlevingsopbouwwerk,1988.0,7.0,4.0,1,NaN,NaN,...,1,0,0,0,0,0,0,0,1,0


## Text pre-processing

In [261]:
#identify text columns
text_columns = ['HN45', 'H_NAAM_VOL', 'Bedrijfsomschrijving']

def preprocess_text(text):
    #tokenization
    tokens = word_tokenize(text)
    
    #normalization (turn into lower case)
    tokens = [token.lower() for token in tokens]
    
    #remove URLs, emojis, numbers etc.
    tokens = [re.sub(r'http\S+', '', token) for token in tokens]  # Remove URLs
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]  # Remove punctuation
    tokens = [token for token in tokens if token.isalpha()]  # Remove numbers
    
    #remove stop words
    stop_words = set(stopwords.words('dutch'))  # You can replace 'english' with the appropriate language
    tokens = [token for token in tokens if token not in stop_words]
    
    #unicode normalization
    tokens = [unicodedata.normalize('NFKD', token).encode('ascii', 'ignore').decode('utf-8') for token in tokens]
    
    #lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

In [96]:
#apply preprocessing to each text column
for col in text_columns:
    df_labelled[col] = df_labelled[col].apply(preprocess_text)

In [156]:
#export to use in classification notebook,( i use the original preprocessedlabelled)
#df_labelled.to_excel('preprocessedlabelledTEST.xlsx', index=False)

### Same procedure for dataset 2022 and 2023

In [262]:
#apply preprocessing to each text column for 2022
for col in text_columns:
    df_KVK2022[col] = df_KVK2022[col].apply(preprocess_text)

In [263]:
#apply preprocessing to each text column for 2022
for col in text_columns:
    df_KVK2023[col] = df_KVK2023[col].apply(preprocess_text)

In [264]:
#export to use in classification notebook
df_KVK2023.to_excel('preprocessedKVK2023TEST.xlsx', index=False)
df_KVK2022.to_excel('preprocessedKVK2022TEST.xlsx', index=False)